In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
%matplotlib inline 
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import export_graphviz
import matplotlib.pyplot as plt
import seaborn as sns

import lightgbm as lgbm
import gc
import xgboost as xgb
import pickle as pickle


from catboost import CatBoostRegressor


In [3]:
items           = pd.read_csv('items.csv',usecols=["item_id", "item_category_id"])
item_categories = pd.read_csv('item_categories.csv')
shops           = pd.read_csv('shops.csv')
sales_train     = pd.read_csv('sales_train.csv.gz')
test            = pd.read_csv('test.csv.gz')

In [4]:
items['item_id'] = pd.to_numeric(items['item_id'],downcast='unsigned')
items['item_category_id'] = pd.to_numeric(items['item_category_id'],downcast='unsigned')

In [5]:
sales_train['date'] = sales_train['date'].astype('category')
sales_train['date_block_num'] = pd.to_numeric(sales_train['date_block_num'],downcast='unsigned')
sales_train['shop_id'] = pd.to_numeric(sales_train['shop_id'],downcast='unsigned')
sales_train['item_price'] = sales_train['item_price'].astype('int')
sales_train['item_price'] = pd.to_numeric(sales_train['item_price'],downcast='unsigned')
sales_train['item_cnt_day'] = pd.to_numeric(sales_train['item_cnt_day'],downcast='signed')


In [6]:
transactions = sales_train
transactions[['day','month', 'year']] = transactions['date'].str.split('.', expand=True).astype(int)
transactions['day'] = pd.to_numeric(transactions['day'],downcast='unsigned')
transactions['month'] = pd.to_numeric(transactions['month'],downcast='unsigned')
transactions['year'] = pd.to_numeric(transactions['year'],downcast='unsigned')


In [7]:
transactions = transactions.set_index('item_id').join(items.set_index('item_id'))
transactions.reset_index(inplace=True)
transactions = transactions[transactions['year'] != 2013]
transactions['y'] = pd.to_numeric(transactions.groupby(['date_block_num', 'item_id', 'shop_id'])['item_cnt_day'].transform('sum')\
                .clip(0,20), downcast='unsigned')

In [8]:
transactions['y'].dtype

dtype('uint8')

In [9]:
len(transactions)

1668287

In [10]:
transactions['turnover'] = transactions['item_price'] * transactions['item_cnt_day']
transactions['turnover'] = pd.to_numeric(transactions['turnover'], downcast='unsigned')

In [11]:
transactions['item_first_block'] = pd.to_numeric(transactions.groupby('item_id')['date_block_num'].transform(np.min), downcast='unsigned')
transactions['item_last_block'] = pd.to_numeric(transactions.groupby('item_id')['date_block_num'].transform(np.max), downcast='unsigned')

transactions['is_first_two_blocks'] = \
                    transactions['date_block_num'].isin([transactions['item_first_block']+1,transactions['item_first_block']+2])


transactions['is_last_two_blocks'] = \
                transactions['date_block_num'].isin([transactions['item_last_block']-1,transactions['item_last_block']])
                                

In [12]:
number_of_items = transactions['item_id'].nunique()
print("number_of_items:", number_of_items)
number_of_categories = transactions['item_category_id'].nunique()
print("number_of_categories:", number_of_categories)
number_of_shops = transactions['shop_id'].nunique()
print("number_of_shops:", number_of_shops)
number_of_days = 365 + 365 - 30 - 31
print("number_of_days:", number_of_days)
number_of_blocks = transactions['date_block_num'].nunique()
print("number_of_blocks:", number_of_blocks)
total_sales = transactions['item_cnt_day'].sum()
print("total_sales:", total_sales)
total_turnover = transactions['turnover'].sum()
print("total_turnover:", total_turnover)
average_price = transactions['item_price'].mean()
print("average_price:", average_price)

number_of_items: 17054
number_of_categories: 79
number_of_shops: 55
number_of_days: 669
number_of_blocks: 22
total_sales: 2085473
total_turnover: 2181307117
average_price: 1015.4701882829513


#ITEM

-UNITS
item_units
item_block_units
item_mean_units_block
item_day_units
item_mean_units_day
item_max_units_block
item_min_units_block
item_max_units_day
item_min_units_day

-TURNOVER
item_turnover
item_block_turnover
item_mean_turnover_block
item_day_turnover
item_mean_turnover_day
item_max_turnover_block
item_min_turnover_block
item_max_turnover_day
item_min_turnover_day


-TIME
item_days_of_activity
item_blocks_of_activity
item_mean_day_between_activity
item_longest_stretch_days_without_activity
item_longest_stretch_blocks_without_activity
item_longest_stretch_block_with_activity
item_number_of_consecutive_days_with_activity
item_days_between_start_and_first_activity
item_blocks_between_start_and_first_activity
item_first_block
item_last_block
item_first_day
item_last_day
item_activity_on_all_blocks


-PRICE
item_mean_price
item_mean_price_block
item_min_price
item_max_price
item_number_different_prices
item_price_amplitude (%age min/max)
item_deviation_mean_category_price


-TREND
is_first_two_full_blocks (actually second/third to make sure we have a "full" block if this was a new release !!!!
is_last_two_blocks
item_first_two_blocks_units
item_last_two_blocks_units
item_fluctuation_units_first_last_blocks
item_first_two_blocks_mean_price
item_last_two_blocks_mean_price
item_fluctuation_price_first_last_blocks

-ENCODINGS
item_share_of_total_units
item_share_of_total_gross
item_share_of_category_units
item_share_of_category_turnover

In [13]:
gc.collect()
transactions_items = transactions.copy()
transactions_items_blocks = transactions.copy()

In [14]:
transactions_items_blocks['item_block_units'] = pd.to_numeric(transactions_items_blocks.groupby(['item_id','date_block_num'])['item_cnt_day'].transform(np.sum), downcast='unsigned')    
transactions_items_blocks['item_block_turnover'] = pd.to_numeric(transactions_items_blocks.groupby(['item_id','date_block_num'])['turnover'].transform(np.sum), downcast='unsigned')    
transactions_items_blocks['item_mean_price_block'] = pd.to_numeric(transactions_items_blocks.groupby(['item_id', 'date_block_num'])['item_price'].transform(np.mean), downcast='float')    

In [15]:
transactions_items['item_units'] = pd.to_numeric(transactions_items.groupby(['item_id'])['item_cnt_day'].transform(np.sum), downcast='unsigned') 
transactions_items['item_mean_units_block'] = pd.to_numeric(transactions_items_blocks.groupby(['item_id'])['item_block_units'].transform(np.mean), downcast='float') 
transactions_items['item_day_units'] = pd.to_numeric(transactions_items.groupby(['item_id','date'])['item_cnt_day'].transform(np.sum), downcast='unsigned') 
transactions_items['item_mean_units_day'] = pd.to_numeric(transactions_items.groupby(['item_id'])['item_day_units'].transform(np.mean), downcast='float') 
transactions_items['item_max_units_block'] = pd.to_numeric(transactions_items_blocks.groupby(['item_id'])['item_block_units'].transform(np.max), downcast='unsigned') 
transactions_items['item_min_units_block'] = pd.to_numeric(transactions_items_blocks.groupby(['item_id'])['item_block_units'].transform(np.min), downcast='unsigned') 
transactions_items['item_max_units_day'] = pd.to_numeric(transactions_items.groupby(['item_id'])['item_day_units'].transform(np.max), downcast='unsigned') 
transactions_items['item_min_units_day'] = pd.to_numeric(transactions_items.groupby(['item_id'])['item_day_units'].transform(np.min), downcast='unsigned') 

In [16]:
transactions_items['item_turnover'] = pd.to_numeric(transactions_items.groupby(['item_id'])['turnover'].transform(np.sum), downcast='unsigned') 
transactions_items['item_mean_turnover_block'] = pd.to_numeric(transactions_items_blocks.groupby(['item_id'])['item_block_turnover'].transform(np.mean), downcast='float') 
transactions_items['item_day_turnover'] = pd.to_numeric(transactions_items.groupby(['item_id','date'])['turnover'].transform(np.sum), downcast='unsigned') 
transactions_items['item_mean_turnover_day'] = pd.to_numeric(transactions_items.groupby(['item_id'])['turnover'].transform(np.mean), downcast='float') 
transactions_items['item_max_turnover_block'] = pd.to_numeric(transactions_items_blocks.groupby(['item_id'])['item_block_turnover'].transform(np.max), downcast='unsigned') 
transactions_items['item_min_turnover_block'] = pd.to_numeric(transactions_items_blocks.groupby(['item_id'])['item_block_turnover'].transform(np.min), downcast='unsigned') 
transactions_items['item_max_turnover_day'] = pd.to_numeric(transactions_items.groupby(['item_id'])['item_day_turnover'].transform(np.max), downcast='unsigned') 
transactions_items['item_min_turnover_day'] = pd.to_numeric(transactions_items.groupby(['item_id'])['item_day_turnover'].transform(np.min), downcast='unsigned') 

In [17]:
transactions_items['item_days_of_activity'] = pd.to_numeric(transactions_items.groupby(['item_id'])['date'].transform("nunique"), downcast='unsigned') 
transactions_items['item_blocks_of_activity'] = pd.to_numeric(transactions_items.groupby(['item_id'])['date_block_num'].transform("nunique"), downcast='unsigned') 

def get_number_of_days_since_start(day,month, year):
    days = 0
    if year == 2015:
        days = 365
    def is_even(num):
        return num % 2 == 0
    half_of_month = int(month/2)
    even = (30*half_of_month) + (31*half_of_month)
    if is_even(month):
        days = days + even - 30 - day
    else:
        days = days + even + day
    return days

transactions_items['item_days_since_start'] = pd.to_numeric(transactions_items.apply(lambda row: get_number_of_days_since_start(row['day'],row['month'], row['year']),axis=1), downcast='unsigned') 

def get_average_days_between_sales(days):
    days = sorted(np.unique(days))
    if len(days) == 0:
        return 9999
    if len(days) == 1:
        return 999
    return np.mean(np.ediff1d(days)) / len(days)

average_days_between_sales = transactions_items.groupby(['item_id'])['item_days_since_start'].apply(list).apply(lambda x: get_average_days_between_sales(x))

transactions_items['item_mean_day_between_activity'] = pd.to_numeric(transactions_items['item_id'].map(average_days_between_sales), downcast='unsigned') 


def get_max_stretch_without_sales_days(days):
    days = np.unique(days)
    max_stretch = 0
    len_days = len(days)
    for index,day in enumerate(sorted(days)):
        if index == len_days - 1:
            return max_stretch
        next_day = days[index+1]
        stretch = next_day - day
        if stretch > max_stretch:
            max_stretch = stretch
            

        
max_stretch_without_sales_day = transactions_items.groupby(['item_id'])['item_days_since_start'].apply(list).apply(lambda x: get_max_stretch_without_sales_days(x))

transactions_items['item_longest_stretch_days_without_activity'] = pd.to_numeric(transactions_items['item_id'].map(max_stretch_without_sales_day), downcast='unsigned') 

In [18]:
gc.collect()

def get_max_stretch_without_sales_block(blocks):
    blocks = np.unique(blocks)
    max_stretch = 0
    len_blocks = len(blocks)
    for index,block in enumerate(sorted(blocks)):
        if index == len_blocks - 1:
            return max_stretch
        next_block = blocks[index+1]
        stretch = next_block - block
        if stretch > max_stretch:
            max_stretch = stretch
            

        
item_longest_stretch_blocks_without_activity = transactions_items.groupby(['item_id'])['date_block_num']\
                                    .apply(list).apply(lambda x: get_max_stretch_without_sales_block(x))

transactions_items['item_longest_stretch_blocks_without_activity'] = pd.to_numeric(transactions_items['item_id'].map(item_longest_stretch_blocks_without_activity), downcast='unsigned') 



def get_longest_stretch(following_pairs, n=1,new_n=1):
    #print("following_pairs", following_pairs, " n: ", n, " new_n: ", new_n)
    len_pairs = len(following_pairs)
    if len_pairs == 0:
        return 0
    if len_pairs == 1:
        if new_n > n:
            return new_n
        return n
    if following_pairs[1][0] == following_pairs[0][1]:
        new_n+=1
    else:
        if new_n > n:
            n=new_n
        new_n=1
    return get_longest_stretch(following_pairs[1:], n,new_n)


assert(get_longest_stretch([]) == 0)
assert(get_longest_stretch([[1, 2], [2,3], [3, 4], [4,5] ,[8,9], [11, 12]]) == 4)
assert(get_longest_stretch([[-1, 0],[1, 2], [2,3], [3, 4], [4,5] ,[8,9], [11, 12]]) == 4)
assert(get_longest_stretch([[1, 2], [4,5] ,[8,9], [9,10],[10, 11]]) == 3)
assert(get_longest_stretch([[1, 2], [4,5] ,[8,9], [9,10],[10, 11],[20, 21], [25,26]]) == 3)
assert(get_longest_stretch([[1, 2], [4,5] ,[8,9], [9,10],[10, 11],[14, 15], [15,16]]) == 3)
assert(get_longest_stretch([[1, 2], [4,5] ,[8,9], [9,10],[10, 11],[14, 15], [15,16],[18,19] ,[22,23], [23,24],[24, 25]]) == 3)
assert(get_longest_stretch([[1, 2], [4,5], [7, 8]]) == 1)
assert(get_longest_stretch([[1, 2], [5, 6], [6, 7], [7,8], [14, 15]]) == 3)
assert(get_longest_stretch([[1, 2], [5, 6], [6, 7], [7,8], [14, 15], [15, 16]]) == 3)
assert(get_longest_stretch([[1, 2], [5, 6], [6, 7], [7,8], [14, 15], [15, 16], [16, 17]]) == 3)
assert(get_longest_stretch([[1, 2], [5, 6], [6, 7], [14, 15], [15, 16], [16, 17]]) == 3)
assert(get_longest_stretch([[1, 2], [5, 6], [6, 7], [7,8], [14, 15], [20, 21], [21, 22], [22,23],[23,24]]) == 4)


def get_following_pairs(pairs):
    pairs = np.unique(pairs)
    len_pairs = len(pairs)
    following = []
    for index,pair in enumerate(sorted(pairs)):
        if index == len_pairs - 1:
            return following
        next_pair = pairs[index+1]
        if next_pair == pair + 1:
            following.append([pair, next_pair])
        
assert(get_following_pairs([1,2,5,6,7,8,9,11,12,15]) == [[1, 2], [5, 6], [6, 7], [7, 8], [8, 9], [11, 12]])
assert(get_following_pairs([1,2,5,6,7,10]) == [[1, 2], [5, 6], [6, 7]])
assert(get_following_pairs([1,2,4,5,7,9,10]) == [[1, 2], [4, 5], [9,10]])
assert(get_following_pairs([1,2,4,5,7,9,10,11,12,15]) == [[1, 2], [4, 5], [9,10],[10,11],[11,12]])


item_longest_stretch_block_with_activity = transactions_items.groupby(['item_id'])['date_block_num']\
                                    .apply(list).apply(lambda x: get_longest_stretch(get_following_pairs(x)))

transactions_items['item_longest_stretch_block_with_activity'] = pd.to_numeric(transactions_items['item_id'].map(item_longest_stretch_block_with_activity), downcast='unsigned') 


item_number_of_consecutive_days_with_activity = transactions_items.groupby(['item_id'])['item_days_since_start']\
                                    .apply(list).apply(lambda x: len(get_following_pairs(x)))
    
transactions_items['item_number_of_consecutive_days_with_activity'] = pd.to_numeric(transactions_items['item_id'].map(item_number_of_consecutive_days_with_activity), downcast='unsigned') 

In [19]:
def get_units_between_first_and_last(units):
    return np.max(units) - np.min(units)

item_days_between_start_and_first_activity = transactions_items.groupby(['item_id'])['item_days_since_start'].apply(list).apply(lambda x: get_units_between_first_and_last(x))
transactions_items['item_days_between_start_and_first_activity'] = pd.to_numeric(transactions_items['item_id'].map(item_days_between_start_and_first_activity), downcast='unsigned') 

item_blocks_between_start_and_first_activity = transactions_items.groupby(['item_id'])['date_block_num'].apply(list).apply(lambda x: get_units_between_first_and_last(x))
transactions_items['item_blocks_between_start_and_first_activity'] = pd.to_numeric(transactions_items['item_id'].map(item_blocks_between_start_and_first_activity), downcast='unsigned') 

In [20]:

transactions_items['item_first_day'] = pd.to_numeric(transactions_items.groupby('item_id')['item_days_since_start'].transform(np.min), downcast='unsigned') 
transactions_items['item_last_day'] = pd.to_numeric(transactions_items.groupby('item_id')['item_days_since_start'].transform(np.max), downcast='unsigned') 

item_activity_on_all_blocks = transactions_items.groupby('item_id')['date_block_num'].nunique().apply(lambda x: x==number_of_blocks)
transactions_items['item_activity_on_all_blocks'] = transactions_items['item_id'].map(item_activity_on_all_blocks)

In [21]:
transactions_items['item_mean_price'] = pd.to_numeric(transactions_items.groupby('item_id')['item_price'].transform(np.mean), downcast='float') 
transactions_items['item_min_price'] = pd.to_numeric(transactions_items.groupby('item_id')['item_price'].transform(np.min), downcast='unsigned') 
transactions_items['item_max_price'] = pd.to_numeric(transactions_items.groupby('item_id')['item_price'].transform(np.max), downcast='unsigned') 
transactions_items['item_number_different_prices'] = pd.to_numeric(transactions_items.groupby('item_id')['item_price'].transform('nunique'), downcast='unsigned') 
transactions_items['item_price_amplitude'] = pd.to_numeric(((transactions_items['item_max_price'] - transactions_items['item_min_price'] ) / transactions_items['item_min_price']) * 100, downcast='float') 
transactions_items['category_mean_price'] = pd.to_numeric(transactions_items.groupby('item_category_id')['item_price'].transform(np.mean), downcast='float') 
transactions_items['item_deviation_mean_category_price'] =  pd.to_numeric(((transactions_items['item_mean_price'] - transactions_items['category_mean_price'] ) / transactions_items['category_mean_price']) * 100, downcast='float') 

In [22]:
item_first_two_blocks_units = transactions_items.groupby(['item_id','is_first_two_blocks'], as_index=False)['item_cnt_day'].sum()
item_first_two_blocks_units = item_first_two_blocks_units[item_first_two_blocks_units['is_first_two_blocks'] == True].set_index('item_id').iloc[:,1]
transactions_items['item_first_two_blocks_units'] = pd.to_numeric(transactions_items['item_id'].map(item_first_two_blocks_units), downcast='unsigned') 

item_last_two_blocks_units = transactions_items.groupby(['item_id','is_last_two_blocks'], as_index=False)['item_cnt_day'].sum()
item_last_two_blocks_units = item_last_two_blocks_units[item_last_two_blocks_units['is_last_two_blocks'] == True].set_index('item_id').iloc[:,1]
transactions_items['item_last_two_blocks_units'] = pd.to_numeric(transactions_items['item_id'].map(item_last_two_blocks_units), downcast='unsigned') 

transactions_items['item_fluctuation_units_first_last_blocks'] =  pd.to_numeric(((transactions_items['item_first_two_blocks_units'] - transactions_items['item_last_two_blocks_units'] ) / \
                                                             transactions_items['item_first_two_blocks_units']) * 100 * -1, downcast='float') 


item_first_two_blocks_mean_price = transactions_items.groupby(['item_id','is_first_two_blocks'], as_index=False)['item_price'].mean()
item_first_two_blocks_mean_price = item_first_two_blocks_mean_price[item_first_two_blocks_mean_price['is_first_two_blocks'] == True].set_index('item_id').iloc[:,1]
transactions_items['item_first_two_blocks_mean_price'] = pd.to_numeric(transactions_items['item_id'].map(item_first_two_blocks_mean_price), downcast='unsigned') 

item_last_two_blocks_mean_price = transactions_items.groupby(['item_id','is_last_two_blocks'], as_index=False)['item_price'].mean()
item_last_two_blocks_mean_price = item_last_two_blocks_mean_price[item_last_two_blocks_mean_price['is_last_two_blocks'] == True].set_index('item_id').iloc[:,1]
transactions_items['item_last_two_blocks_mean_price'] = pd.to_numeric(transactions_items['item_id'].map(item_last_two_blocks_mean_price), downcast='unsigned') 

transactions_items['item_fluctuation_price_first_last_blocks'] =  pd.to_numeric(((transactions_items['item_first_two_blocks_mean_price'] - transactions_items['item_last_two_blocks_mean_price'] ) / \
                                                             transactions_items['item_first_two_blocks_mean_price'])  * 100 * -1, downcast='float') 

In [23]:
transactions_items['item_share_of_total_units'] = pd.to_numeric(transactions_items['item_units'] * 100 / total_sales , downcast='float') 

transactions_items['item_share_of_total_turnover'] = pd.to_numeric(transactions_items['item_turnover'] * 100 / total_turnover, downcast='float') 

transactions_items['category_units'] = pd.to_numeric(transactions_items.groupby('item_category_id')['item_cnt_day'].transform(np.sum), downcast='unsigned') 
transactions_items['item_share_of_category_units'] = pd.to_numeric(transactions_items['item_units'] * 100 / transactions_items['category_units'], downcast='float') 

transactions_items['category_turnover'] = pd.to_numeric(transactions_items.groupby('item_category_id')['turnover'].transform(np.sum), downcast='unsigned') 
transactions_items['item_share_of_category_turnover'] = pd.to_numeric(transactions_items['item_turnover'] * 100 / transactions_items['category_turnover'], downcast='float') 


In [24]:
transactions_items.to_pickle("pickled/transactions_items")
transactions_items_blocks.to_pickle("pickled/transactions_items_blocks")

del transactions_items
del transactions_items_blocks
gc.collect()

336

#CATEGORY

-UNITS
category_units
category_block_units
category_mean_units_block
category_day_units
category_mean_units_day
category_max_units_block
category_min_units_block
category_max_units_day
category_min_units_day

-TURNOVER
category_turnover
category_block_turnover
category_mean_turnover_block
category_day_turnover
category_mean_turnover_day
category_max_turnover_block
category_min_turnover_block
category_max_turnover_day
category_min_turnover_day


-PRICE
category_mean_price
category_mean_price_block
category_min_price
category_max_price


-TREND
category_first_two_blocks_units
category_last_two_blocks_units
category_fluctuation_units_first_last_blocks
category_first_two_blocks_mean_price
category_last_two_blocks_mean_price
category_fluctuation_price_first_last_blocks

-SUBCATEGORY
subcategory
subcategory 1hot

-UNITS
subcategory_units
subcategory_block_units
subcategory_mean_units_block
subcategory_day_units
subcategory_mean_units_day
subcategory_max_units_block
subcategory_min_units_block
subcategory_max_units_day
subcategory_min_units_day

-TURNOVER
subcategory_turnover
subcategory_block_turnover
subcategory_mean_turnover_block
subcategory_day_turnover
subcategory_mean_turnover_day
subcategory_max_turnover_block
subcategory_min_turnover_block
subcategory_max_turnover_day
subcategory_min_turnover_day

-ENCODINGS
category_share_of_total_units
category_share_of_total_gross
subcategory_share_of_total_units
subcategory_share_of_total_gross

-TREND
subcategory_first_two_blocks_units
subcategory_last_two_blocks_units
subcategory_fluctuation_units_first_last_blocks
subcategory_first_two_blocks_mean_price
subcategory_last_two_blocks_mean_price
subcategory_fluctuation_price_first_last_blocks

In [25]:
gc.collect()
transactions_categories = transactions.copy()
transactions_categories_blocks = transactions.copy()

In [26]:
sub_cats = {}
for i in range(1,8):
    sub_cats[i] = "Accessories"
sub_cats[8] = "Tickets"
sub_cats[9] = "Delivery of goods"
for i in range(10,18):
    sub_cats[i] = "Consoles"
for i in range(18,25):
    sub_cats[i] = "Game for Consoles"
sub_cats[25] = "Accessories for Games"
sub_cats[26] = "Android Games"
sub_cats[27] = "MAC Games"
for i in range(28,32):
    sub_cats[i] = "PC Games"
for i in range(32,37):
    sub_cats[i] = "Payment Cards"
for i in range(37,40):
    sub_cats[i] = "Cinema - Blu-ray"
sub_cats[40] = "Cinema - DVD"
sub_cats[41] = "Cinema - Collectible"
for i in range(42,46):
    sub_cats[i] = "Audiobooks"
for i in range(46,55):
    sub_cats[i] = "Books"
for i in range(55,57):
    sub_cats[i] = "Music - CD"
sub_cats[57] = "Music - MP3"
sub_cats[58] = "Music - Vinyl"
sub_cats[59] = "Music - Music Video"
sub_cats[60] = "Music - Gift Edition"
for i in range(61,74):
    sub_cats[i] = "Gifts"
for i in range(73,79):
    sub_cats[i] = "Software"
sub_cats[79] = "Utility"
for i in range(80,84):
    sub_cats[i] = "Misc"
    
    
transactions_categories['subcategory'] = transactions_categories['item_category_id'].apply(lambda x: sub_cats[x]).astype('category')
transactions_categories_blocks['subcategory'] = transactions_categories_blocks['item_category_id'].apply(lambda x: sub_cats[x]).astype('category')

In [27]:
transactions_categories_blocks['category_block_units'] = pd.to_numeric(transactions_categories_blocks.groupby(['item_category_id','date_block_num'])['item_cnt_day'].transform(np.sum), downcast='unsigned') 
transactions_categories_blocks['category_block_turnover'] = pd.to_numeric(transactions_categories_blocks.groupby(['item_category_id','date_block_num'])['turnover'].transform(np.sum), downcast='unsigned') 
transactions_categories_blocks['category_mean_price_block'] = pd.to_numeric(transactions_categories_blocks.groupby(['item_category_id', 'date_block_num'])['item_price'].transform(np.mean), downcast='float') 
transactions_categories_blocks['subcategory_block_units'] = pd.to_numeric(transactions_categories_blocks.groupby(['subcategory','date_block_num'])['item_cnt_day'].transform(np.sum), downcast='unsigned') 
transactions_categories_blocks['subcategory_block_turnover'] = pd.to_numeric(transactions_categories_blocks.groupby(['subcategory','date_block_num'])['turnover'].transform(np.sum), downcast='unsigned') 
transactions_categories_blocks['subcategory_mean_price_block'] = pd.to_numeric(transactions_categories_blocks.groupby(['subcategory', 'date_block_num'])['item_price'].transform(np.mean), downcast='float') 


In [28]:
transactions_categories['category_units'] = pd.to_numeric(transactions_categories.groupby(['item_category_id'])['item_cnt_day'].transform(np.sum), downcast='unsigned') 
transactions_categories['category_mean_units_block'] = pd.to_numeric(transactions_categories_blocks.groupby(['item_category_id'])['category_block_units'].transform(np.mean), downcast='float') 
transactions_categories['category_day_units'] = pd.to_numeric(transactions_categories.groupby(['item_category_id','date'])['item_cnt_day'].transform(np.sum), downcast='unsigned') 
transactions_categories['category_mean_units_day'] = pd.to_numeric(transactions_categories.groupby(['item_category_id'])['category_day_units'].transform(np.mean), downcast='float') 
transactions_categories['category_max_units_block'] = pd.to_numeric(transactions_categories_blocks.groupby(['item_category_id'])['category_block_units'].transform(np.max), downcast='unsigned') 
transactions_categories['category_min_units_block'] = pd.to_numeric(transactions_categories_blocks.groupby(['item_category_id'])['category_block_units'].transform(np.min), downcast='unsigned') 
transactions_categories['category_max_units_day'] = pd.to_numeric(transactions_categories.groupby(['item_category_id'])['category_day_units'].transform(np.max), downcast='unsigned') 
transactions_categories['category_min_units_day'] = pd.to_numeric(transactions_categories.groupby(['item_category_id'])['category_day_units'].transform(np.min), downcast='unsigned') 

In [29]:
transactions_categories['category_turnover'] = pd.to_numeric(transactions_categories.groupby(['item_category_id'])['turnover'].transform(np.sum), downcast='unsigned') 
transactions_categories['category_mean_turnover_block'] = pd.to_numeric(transactions_categories_blocks.groupby(['item_category_id'])['category_block_turnover'].transform(np.mean), downcast='float') 
transactions_categories['category_day_turnover'] = pd.to_numeric(transactions_categories.groupby(['item_category_id','date'])['turnover'].transform(np.sum), downcast='unsigned') 
transactions_categories['category_mean_turnover_day'] = pd.to_numeric(transactions_categories.groupby(['item_category_id'])['category_day_turnover'].transform(np.mean), downcast='float') 
transactions_categories['category_max_turnover_block'] = pd.to_numeric(transactions_categories_blocks.groupby(['item_category_id'])['category_block_turnover'].transform(np.max), downcast='unsigned') 
transactions_categories['category_min_turnover_block'] = pd.to_numeric(transactions_categories_blocks.groupby(['item_category_id'])['category_block_turnover'].transform(np.min), downcast='unsigned') 
transactions_categories['category_max_turnover_day'] = pd.to_numeric(transactions_categories.groupby(['item_category_id'])['category_day_turnover'].transform(np.max), downcast='unsigned') 
transactions_categories['category_min_turnover_day'] = pd.to_numeric(transactions_categories.groupby(['item_category_id'])['category_day_turnover'].transform(np.min), downcast='unsigned') 

In [30]:
transactions_categories['category_mean_price'] = pd.to_numeric(transactions_categories.groupby('item_category_id')['item_price'].transform(np.mean), downcast='float') 
transactions_categories['category_min_price'] = pd.to_numeric(transactions_categories.groupby('item_category_id')['item_price'].transform(np.min), downcast='unsigned')
transactions_categories['category_max_price'] = pd.to_numeric(transactions_categories.groupby('item_category_id')['item_price'].transform(np.max), downcast='unsigned')

In [31]:
category_first_two_blocks_units = transactions_categories.groupby(['item_category_id','is_first_two_blocks'], as_index=False)['item_cnt_day'].sum()
category_first_two_blocks_units = category_first_two_blocks_units[category_first_two_blocks_units['is_first_two_blocks'] == True].set_index('item_category_id').iloc[:,1]
transactions_categories['category_first_two_blocks_units'] = pd.to_numeric(transactions_categories['item_category_id'].map(category_first_two_blocks_units), downcast='unsigned')

category_last_two_blocks_units = transactions_categories.groupby(['item_category_id','is_last_two_blocks'], as_index=False)['item_cnt_day'].sum()
category_last_two_blocks_units = category_last_two_blocks_units[category_last_two_blocks_units['is_last_two_blocks'] == True].set_index('item_category_id').iloc[:,1]
transactions_categories['category_last_two_blocks_units'] = pd.to_numeric(transactions_categories['item_category_id'].map(category_last_two_blocks_units), downcast='unsigned')

transactions_categories['category_fluctuation_units_first_last_blocks'] =  pd.to_numeric(((transactions_categories['category_first_two_blocks_units'] - transactions_categories['category_last_two_blocks_units'] ) / \
                                                             transactions_categories['category_first_two_blocks_units']) * 100 * -1, downcast='float') 


category_first_two_blocks_mean_price = transactions_categories.groupby(['item_category_id','is_first_two_blocks'], as_index=False)['item_price'].mean()
category_first_two_blocks_mean_price = category_first_two_blocks_mean_price[category_first_two_blocks_mean_price['is_first_two_blocks'] == True].set_index('item_category_id').iloc[:,1]
transactions_categories['category_first_two_blocks_mean_price'] = pd.to_numeric(transactions_categories['item_category_id'].map(category_first_two_blocks_mean_price), downcast='unsigned')

category_last_two_blocks_mean_price = transactions_categories.groupby(['item_category_id','is_last_two_blocks'], as_index=False)['item_price'].mean()
category_last_two_blocks_mean_price = category_last_two_blocks_mean_price[category_last_two_blocks_mean_price['is_last_two_blocks'] == True].set_index('item_category_id').iloc[:,1]
transactions_categories['category_last_two_blocks_mean_price'] = pd.to_numeric(transactions_categories['item_category_id'].map(category_last_two_blocks_mean_price), downcast='unsigned')

transactions_categories['category_fluctuation_price_first_last_blocks'] =  pd.to_numeric(((transactions_categories['category_first_two_blocks_mean_price'] - transactions_categories['category_last_two_blocks_mean_price'] ) / \
                                                             transactions_categories['category_first_two_blocks_mean_price'])  * 100 * -1, downcast='float') 

In [32]:
transactions_categories['video_game'] = transactions_categories["item_category_id"].isin(list(range(18,32)))
transactions_categories['gaming_old_gen'] = transactions_categories["item_category_id"].isin([10,11,15,18,19,23])
transactions_categories['gaming_new_gen'] = transactions_categories["item_category_id"].isin([12,14,16,20,22,24])
transactions_categories['pc_games'] = transactions_categories["item_category_id"].isin(list(range(27,32)))
transactions_categories['payment_cards'] = transactions_categories["item_category_id"].isin(list(range(32,37)))
transactions_categories['movies'] = transactions_categories["item_category_id"].isin(list(range(37,42)))
transactions_categories['movies_niche'] = transactions_categories["item_category_id"].isin([38,39])
transactions_categories['books'] = transactions_categories["item_category_id"].isin([42,55])
transactions_categories['music'] = transactions_categories["item_category_id"].isin(list(range(55,61)))
transactions_categories['music_CD'] = transactions_categories["item_category_id"].isin([55,56])
transactions_categories['music_vinyl'] = transactions_categories["item_category_id"].isin([58])
transactions_categories['gifts'] = transactions_categories["item_category_id"].isin(list(range(61,72)))
transactions_categories['software'] = transactions_categories["item_category_id"].isin(list(range(73,79)))

In [33]:
transactions_categories['subcategory_units'] = pd.to_numeric(transactions_categories.groupby(['subcategory'])['item_cnt_day'].transform(np.sum), downcast='unsigned')
transactions_categories['subcategory_mean_units_block'] = pd.to_numeric(transactions_categories_blocks.groupby(['subcategory'])['subcategory_block_units'].transform(np.mean), downcast='float') 
transactions_categories['subcategory_day_units'] = pd.to_numeric(transactions_categories.groupby(['subcategory','date'])['item_cnt_day'].transform(np.sum), downcast='unsigned')
transactions_categories['subcategory_mean_units_day'] = pd.to_numeric(transactions_categories.groupby(['subcategory'])['subcategory_day_units'].transform(np.mean), downcast='float') 
transactions_categories['subcategory_max_units_block'] = pd.to_numeric(transactions_categories_blocks.groupby(['subcategory'])['subcategory_block_units'].transform(np.max), downcast='unsigned')
transactions_categories['subcategory_min_units_block'] = pd.to_numeric(transactions_categories_blocks.groupby(['subcategory'])['subcategory_block_units'].transform(np.min), downcast='unsigned')
transactions_categories['subcategory_max_units_day'] = pd.to_numeric(transactions_categories.groupby(['subcategory'])['subcategory_day_units'].transform(np.max), downcast='unsigned')
transactions_categories['subcategory_min_units_day'] = pd.to_numeric(transactions_categories.groupby(['subcategory'])['subcategory_day_units'].transform(np.min), downcast='unsigned')

In [34]:
transactions_categories['subcategory_turnover'] = pd.to_numeric(transactions_categories.groupby(['subcategory'])['turnover'].transform(np.sum), downcast='unsigned')
transactions_categories['subcategory_mean_turnover_block'] = pd.to_numeric(transactions_categories_blocks.groupby(['subcategory'])['subcategory_block_turnover'].transform(np.mean), downcast='float') 
transactions_categories['subcategory_day_turnover'] = pd.to_numeric(transactions_categories.groupby(['subcategory','date'])['turnover'].transform(np.sum), downcast='unsigned')
transactions_categories['subcategory_mean_turnover_day'] = pd.to_numeric(transactions_categories.groupby(['subcategory'])['subcategory_day_turnover'].transform(np.mean), downcast='float') 
transactions_categories['subcategory_max_turnover_block'] = pd.to_numeric(transactions_categories_blocks.groupby(['subcategory'])['subcategory_block_turnover'].transform(np.max), downcast='unsigned')
transactions_categories['subcategory_min_turnover_block'] = pd.to_numeric(transactions_categories_blocks.groupby(['subcategory'])['subcategory_block_turnover'].transform(np.min), downcast='unsigned')
transactions_categories['subcategory_max_turnover_day'] = pd.to_numeric(transactions_categories.groupby(['subcategory'])['subcategory_day_turnover'].transform(np.max), downcast='unsigned')
transactions_categories['subcategory_min_turnover_day'] = pd.to_numeric(transactions_categories.groupby(['subcategory'])['subcategory_day_turnover'].transform(np.min), downcast='unsigned')

In [35]:
transactions_categories['category_share_of_total_units'] = pd.to_numeric(transactions_categories['category_units'] * 100 / total_sales , downcast='float') 
transactions_categories['category_share_of_total_turnover'] = pd.to_numeric(transactions_categories['category_turnover']* 100 / total_turnover, downcast='float') 

transactions_categories['subcategory_units'] = pd.to_numeric(transactions_categories.groupby("subcategory")['item_cnt_day'].transform(np.sum), downcast='unsigned')
transactions_categories['subcategory_share_of_total_units'] = pd.to_numeric(transactions_categories['subcategory_units'] * 100 / total_sales, downcast='float') 
transactions_categories['subcategory_turnover'] = pd.to_numeric(transactions_categories.groupby("subcategory")['turnover'].transform(np.sum), downcast='unsigned')
transactions_categories['subcategory_share_of_total_turnover'] = pd.to_numeric(transactions_categories['subcategory_turnover']* 100 / total_turnover, downcast='float') 

In [36]:
subcategory_first_two_blocks_units = transactions_categories.groupby(['subcategory','is_first_two_blocks'], as_index=False)['item_cnt_day'].sum()
subcategory_first_two_blocks_units = subcategory_first_two_blocks_units[subcategory_first_two_blocks_units['is_first_two_blocks'] == True].set_index('subcategory').iloc[:,1]
transactions_categories['subcategory_first_two_blocks_units'] = pd.to_numeric(transactions_categories['subcategory'].map(subcategory_first_two_blocks_units), downcast='unsigned')

subcategory_last_two_blocks_units = transactions_categories.groupby(['subcategory','is_last_two_blocks'], as_index=False)['item_cnt_day'].sum()
subcategory_last_two_blocks_units = subcategory_last_two_blocks_units[subcategory_last_two_blocks_units['is_last_two_blocks'] == True].set_index('subcategory').iloc[:,1]
transactions_categories['subcategory_last_two_blocks_units'] = pd.to_numeric(transactions_categories['subcategory'].map(subcategory_last_two_blocks_units), downcast='unsigned')

transactions_categories['subcategory_fluctuation_units_first_last_blocks'] =  pd.to_numeric(((transactions_categories['subcategory_first_two_blocks_units'] - transactions_categories['subcategory_last_two_blocks_units'] ) / \
                                                             transactions_categories['subcategory_first_two_blocks_units']) * 100 * -1, downcast='float') 


subcategory_first_two_blocks_mean_price = transactions_categories.groupby(['subcategory','is_first_two_blocks'], as_index=False)['item_price'].mean()
subcategory_first_two_blocks_mean_price = subcategory_first_two_blocks_mean_price[subcategory_first_two_blocks_mean_price['is_first_two_blocks'] == True].set_index('subcategory').iloc[:,1]
transactions_categories['subcategory_first_two_blocks_mean_price'] = pd.to_numeric(transactions_categories['subcategory'].map(subcategory_first_two_blocks_mean_price), downcast='unsigned')

subcategory_last_two_blocks_mean_price = transactions_categories.groupby(['subcategory','is_last_two_blocks'], as_index=False)['item_price'].mean()
subcategory_last_two_blocks_mean_price = subcategory_last_two_blocks_mean_price[subcategory_last_two_blocks_mean_price['is_last_two_blocks'] == True].set_index('subcategory').iloc[:,1]
transactions_categories['subcategory_last_two_blocks_mean_price'] = pd.to_numeric(transactions_categories['subcategory'].map(subcategory_last_two_blocks_mean_price), downcast='unsigned')

transactions_categories['subcategory_fluctuation_price_first_last_blocks'] =  pd.to_numeric(((transactions_categories['subcategory_first_two_blocks_mean_price'] - transactions_categories['subcategory_last_two_blocks_mean_price'] ) / \
                                                             transactions_categories['subcategory_first_two_blocks_mean_price'])  * 100 * -1, downcast='float') 

In [37]:

transactions_categories.to_pickle("pickled/transactions_categories")
transactions_categories_blocks.to_pickle("pickled/transactions_categories_blocks")

del transactions_categories
del transactions_categories_blocks
gc.collect()

147

#SHOP

-UNITS
shop_units
shop_block_units
shop_mean_units_block
shop_day_units
shop_mean_units_day
shop_max_units_block
shop_min_units_block
shop_max_units_day
shop_min_units_day

-TURNOVER
shop_turnover
shop_block_turnover
shop_mean_turnover_block
shop_day_turnover
shop_mean_turnover_day
shop_max_turnover_block
shop_min_turnover_block
shop_max_turnover_day
shop_min_turnover_day

-PRICE
shop_mean_price
shop_mean_price_block


-TREND
shop_first_two_blocks_units
shop_last_two_blocks_units
shop_fluctuation_units_first_last_blocks
shop_first_two_blocks_mean_price
shop_last_two_blocks_mean_price
shop_fluctuation_price_first_last_blocks

-ENCODINGS
shop_share_of_total_units
shop_share_of_total_gross

-MISC
shop_ids_TC
shop_ids_TRK
shop_ids_SEC
shop_ids_shopping_center
shop_ids_moscow

-CATEGORY
shop_top_category_units
shop_top_category_turnover
shop_top_subcategory_units
shop_top_subcategory_turnover

In [38]:
gc.collect()
transactions_shops = transactions.copy()
transactions_shops_blocks = transactions.copy()

In [39]:
shop_areas = {}
for i in range(0,2):
    shop_areas[i] = "Yakutsk"
shop_areas[2] = "Adygea"
shop_areas[3] = "Balashikha"
shop_areas[4] = "Volga"
shop_areas[5] = "Vologda"
for i in range(6,9):
    shop_areas[i] = "Voronezh"
shop_areas[9] = "Outbound Trading"
for i in range(10,12):
    shop_areas[i] = "Zhukovsky"
shop_areas[12] = "Online store emergency"
for i in range(13,15):
    shop_areas[i] = "Kazan"
shop_areas[15] = "Kaluga"
shop_areas[16] = "Kolomna"
for i in range(17,19):
    shop_areas[i] = "Krasnoyarsk"
shop_areas[19] = "Kursk"
for i in range(20,33):
    shop_areas[i] = "Moscow"
shop_areas[33] = "Mytishchi"
for i in range(34,36):
    shop_areas[i] = "N.Novgorod"
for i in range(36,38):
    shop_areas[i] = "Novosibirsk"
shop_areas[38] = "Omsk"
for i in range(39,42):
    shop_areas[i] = "RostovNaDonu"
for i in range(42,44):
    shop_areas[i] = "St. Petersburg"
for i in range(44,46):
    shop_areas[i] = "Samara"
shop_areas[46] = "Sergiev Posad"
shop_areas[47] = "Surgut"
shop_areas[48] = "Tomsk"
for i in range(49,52):
    shop_areas[i] = "Tyumen TC"
for i in range(52,54):
    shop_areas[i] = "Ufa"
shop_areas[54] = "Khimki"
shop_areas[55] = "Digital warehouse"
shop_areas[56] = "Chekhov"
for i in range(57,59):
    shop_areas[i] = "Yakutsk"
shop_areas[59] = "Yaroslavl"

transactions_shops['area'] = transactions_shops['shop_id'].apply(lambda x: shop_areas[x]).astype('category')

transactions_shops_blocks['area'] = transactions_shops['shop_id'].apply(lambda x: shop_areas[x]).astype('category')



In [40]:
transactions_shops_blocks['shop_block_units'] = pd.to_numeric(transactions_shops_blocks.groupby(['shop_id','date_block_num'])['item_cnt_day'].transform(np.sum), downcast='unsigned')
transactions_shops_blocks['shop_block_turnover'] = pd.to_numeric(transactions_shops_blocks.groupby(['shop_id','date_block_num'])['turnover'].transform(np.sum), downcast='unsigned')
transactions_shops_blocks['shop_mean_price_block'] = pd.to_numeric(transactions_shops_blocks.groupby(['shop_id', 'date_block_num'])['item_price'].transform(np.mean), downcast='float') 
transactions_shops_blocks['area_block_units'] = pd.to_numeric(transactions_shops_blocks.groupby(['area','date_block_num'])['item_cnt_day'].transform(np.sum), downcast='unsigned')
transactions_shops_blocks['area_block_turnover'] = pd.to_numeric(transactions_shops_blocks.groupby(['area','date_block_num'])['turnover'].transform(np.sum), downcast='unsigned')
transactions_shops_blocks['area_mean_price_block'] = pd.to_numeric(transactions_shops_blocks.groupby(['area', 'date_block_num'])['item_price'].transform(np.mean), downcast='float') 


In [41]:
transactions_shops['shop_units'] = pd.to_numeric(transactions_shops.groupby(['shop_id'])['item_cnt_day'].transform(np.sum), downcast='unsigned')
transactions_shops['shop_mean_units_block'] = pd.to_numeric(transactions_shops_blocks.groupby(['shop_id'])['shop_block_units'].transform(np.mean), downcast='float') 
transactions_shops['shop_day_units'] = pd.to_numeric(transactions_shops.groupby(['shop_id','date'])['item_cnt_day'].transform(np.sum), downcast='unsigned')
transactions_shops['shop_mean_units_day'] = pd.to_numeric(transactions_shops.groupby(['shop_id'])['shop_day_units'].transform(np.mean), downcast='float') 
transactions_shops['shop_max_units_block'] = pd.to_numeric(transactions_shops_blocks.groupby(['shop_id'])['shop_block_units'].transform(np.max), downcast='unsigned')
transactions_shops['shop_min_units_block'] = pd.to_numeric(transactions_shops_blocks.groupby(['shop_id'])['shop_block_units'].transform(np.min), downcast='unsigned')
transactions_shops['shop_max_units_day'] = pd.to_numeric(transactions_shops.groupby(['shop_id'])['shop_day_units'].transform(np.max), downcast='unsigned')
transactions_shops['shop_min_units_day'] = pd.to_numeric(transactions_shops.groupby(['shop_id'])['shop_day_units'].transform(np.min), downcast='unsigned')

In [42]:
transactions_shops['shop_turnover'] = pd.to_numeric(transactions_shops.groupby(['shop_id'])['turnover'].transform(np.sum), downcast='unsigned')
transactions_shops['shop_mean_turnover_block'] = pd.to_numeric(transactions_shops_blocks.groupby(['shop_id'])['shop_block_turnover'].transform(np.mean), downcast='float') 
transactions_shops['shop_day_turnover'] = pd.to_numeric(transactions_shops.groupby(['shop_id','date'])['turnover'].transform(np.sum), downcast='unsigned')
transactions_shops['shop_mean_turnover_day'] = pd.to_numeric(transactions_shops.groupby(['shop_id'])['shop_day_turnover'].transform(np.mean), downcast='float') 
transactions_shops['shop_max_turnover_block'] = pd.to_numeric(transactions_shops_blocks.groupby(['shop_id'])['shop_block_turnover'].transform(np.max), downcast='unsigned')
transactions_shops['shop_min_turnover_block'] = pd.to_numeric(transactions_shops_blocks.groupby(['shop_id'])['shop_block_turnover'].transform(np.min), downcast='unsigned')
transactions_shops['shop_max_turnover_day'] = pd.to_numeric(transactions_shops.groupby(['shop_id'])['shop_day_turnover'].transform(np.max), downcast='unsigned')
transactions_shops['shop_min_turnover_day'] = pd.to_numeric(transactions_shops.groupby(['shop_id'])['shop_day_turnover'].transform(np.min), downcast='unsigned')

In [43]:
transactions_shops['shop_mean_price'] = pd.to_numeric(transactions_shops.groupby('shop_id')['item_price'].transform(np.mean), downcast='float') 


In [44]:
shop_first_two_blocks_units = transactions_shops.groupby(['shop_id','is_first_two_blocks'], as_index=False)['item_cnt_day'].sum()
shop_first_two_blocks_units = shop_first_two_blocks_units[shop_first_two_blocks_units['is_first_two_blocks'] == True].set_index('shop_id').iloc[:,1]
transactions_shops['shop_first_two_blocks_units'] = pd.to_numeric(transactions_shops['shop_id'].map(shop_first_two_blocks_units), downcast='unsigned')

shop_last_two_blocks_units = transactions_shops.groupby(['shop_id','is_last_two_blocks'], as_index=False)['item_cnt_day'].sum()
shop_last_two_blocks_units = shop_last_two_blocks_units[shop_last_two_blocks_units['is_last_two_blocks'] == True].set_index('shop_id').iloc[:,1]
transactions_shops['shop_last_two_blocks_units'] = pd.to_numeric(transactions_shops['shop_id'].map(shop_last_two_blocks_units), downcast='unsigned')

transactions_shops['shop_fluctuation_units_first_last_blocks'] =  pd.to_numeric(((transactions_shops['shop_first_two_blocks_units'] - transactions_shops['shop_last_two_blocks_units'] ) / \
                                                             transactions_shops['shop_first_two_blocks_units']) * 100 * -1, downcast='float') 


shop_first_two_blocks_mean_price = transactions_shops.groupby(['shop_id','is_first_two_blocks'], as_index=False)['item_price'].mean()
shop_first_two_blocks_mean_price = shop_first_two_blocks_mean_price[shop_first_two_blocks_mean_price['is_first_two_blocks'] == True].set_index('shop_id').iloc[:,1]
transactions_shops['shop_first_two_blocks_mean_price'] = pd.to_numeric(transactions_shops['shop_id'].map(shop_first_two_blocks_mean_price), downcast='unsigned')

shop_last_two_blocks_mean_price = transactions_shops.groupby(['shop_id','is_last_two_blocks'], as_index=False)['item_price'].mean()
shop_last_two_blocks_mean_price = shop_last_two_blocks_mean_price[shop_last_two_blocks_mean_price['is_last_two_blocks'] == True].set_index('shop_id').iloc[:,1]
transactions_shops['shop_last_two_blocks_mean_price'] = pd.to_numeric(transactions_shops['shop_id'].map(shop_last_two_blocks_mean_price), downcast='unsigned')

transactions_shops['shop_fluctuation_price_first_last_blocks'] =  pd.to_numeric(((transactions_shops['shop_first_two_blocks_mean_price'] - transactions_shops['shop_last_two_blocks_mean_price'] ) / \
                                                             transactions_shops['shop_first_two_blocks_mean_price'])  * 100 * -1, downcast='float') 

In [45]:
transactions_shops['shop_share_of_units'] = pd.to_numeric(transactions_shops['shop_units'] * 100 / total_sales, downcast='float') 
transactions_shops['shop_share_of_turnover'] = pd.to_numeric(transactions_shops['shop_turnover'] * 100 / total_turnover, downcast='float') 

In [46]:
shop_ids_TC = [1,2,13,14,16,23,24,26,28,31,37,38,42,43,44,46,50,54,58]
shop_ids_TRK = [3,33,39,40]
shop_ids_SEC = [7,34,36,47,48,49,56]
shop_ids_shopping_center = [4,5,8,15,17,18,19,27,29,30,32,41,45,51,53,59]
shop_ids_moscow = list(range(20,33))


transactions_shops['shop_TC'] = transactions_shops['shop_id'].isin(shop_ids_TC)
transactions_shops['shop_TRK'] = transactions_shops['shop_id'].isin(shop_ids_TRK)
transactions_shops['shop_SEC'] = transactions_shops['shop_id'].isin(shop_ids_SEC)
transactions_shops['shop_shopping_center'] = transactions_shops['shop_id'].isin(shop_ids_shopping_center)
transactions_shops['shop_moscow'] = transactions_shops['shop_id'].isin(shop_ids_moscow)

In [47]:
a = transactions_shops.groupby(['shop_id', 'item_category_id'],as_index=False)['item_cnt_day'].sum()

b = pd.DataFrame(transactions_shops.groupby(['shop_id', 'item_category_id'],as_index=False)['item_cnt_day'].sum()\
                  .groupby(['shop_id'])['item_cnt_day'].max()).reset_index()

c = b.merge(a, on=['shop_id', 'item_cnt_day'],how='left')[['shop_id', 'item_category_id']].rename(columns={'item_category_id':'max_category_units'})

transactions_shops = transactions_shops.merge(c, on=['shop_id'], how='left')

transactions_shops['max_category_units'] = pd.to_numeric(transactions_shops['max_category_units'], downcast='unsigned')

In [48]:
a = transactions_shops.groupby(['shop_id', 'item_category_id'],as_index=False)['turnover'].sum()

b = pd.DataFrame(transactions_shops.groupby(['shop_id', 'item_category_id'],as_index=False)['turnover'].sum()\
                  .groupby(['shop_id'])['turnover'].max()).reset_index()

c = b.merge(a, on=['shop_id', 'turnover'],how='left')[['shop_id', 'item_category_id']].rename(columns={'item_category_id':'max_category_turnover'})

transactions_shops = transactions_shops.merge(c, on=['shop_id'], how='left')
transactions_shops['max_category_turnover'] = pd.to_numeric(transactions_shops['max_category_turnover'], downcast='unsigned')

-AREA
area



-UNITS
area_units
area_block_units
area_mean_units_block
area_day_units
area_mean_units_day
area_max_units_block
area_min_units_block
area_max_units_day
area_min_units_day

-TURNOVER
area_turnover
area_block_turnover
area_mean_turnover_block
area_day_turnover
area_mean_turnover_day
area_max_turnover_block
area_min_turnover_block
area_max_turnover_day
area_min_turnover_day

-PRICE
area_mean_price
area_mean_price_block


-TREND
area_first_two_blocks_units
area_last_two_blocks_units
area_fluctuation_units_first_last_blocks
area_first_two_blocks_mean_price
area_last_two_blocks_mean_price
area_fluctuation_price_first_last_blocks

-ENCODINGS
area_share_of_total_units
area_share_of_total_gross

In [49]:
transactions_shops['area_units'] = pd.to_numeric(transactions_shops.groupby(['area'])['item_cnt_day'].transform(np.sum), downcast='unsigned')
transactions_shops['area_mean_units_block'] = pd.to_numeric(transactions_shops_blocks.groupby(['area'])['area_block_units'].transform(np.mean), downcast='float') 
transactions_shops['area_day_units'] = pd.to_numeric(transactions_shops.groupby(['area','date'])['item_cnt_day'].transform(np.sum), downcast='unsigned')
transactions_shops['area_mean_units_day'] = pd.to_numeric(transactions_shops.groupby(['area'])['area_day_units'].transform(np.mean), downcast='float') 
transactions_shops['area_max_units_block'] = pd.to_numeric(transactions_shops_blocks.groupby(['area'])['area_block_units'].transform(np.max), downcast='unsigned')
transactions_shops['area_min_units_block'] = pd.to_numeric(transactions_shops_blocks.groupby(['area'])['area_block_units'].transform(np.min), downcast='unsigned')
transactions_shops['area_max_units_day'] = pd.to_numeric(transactions_shops.groupby(['area'])['area_day_units'].transform(np.max), downcast='unsigned')
transactions_shops['area_min_units_day'] = pd.to_numeric(transactions_shops.groupby(['area'])['area_day_units'].transform(np.min), downcast='unsigned')

In [50]:
transactions_shops['area_turnover'] = pd.to_numeric(transactions_shops.groupby(['area'])['turnover'].transform(np.sum), downcast='unsigned')
transactions_shops['area_mean_turnover_block'] = pd.to_numeric(transactions_shops_blocks.groupby(['area'])['area_block_turnover'].transform(np.mean), downcast='float') 
transactions_shops['area_day_turnover'] = pd.to_numeric(transactions_shops.groupby(['area','date'])['turnover'].transform(np.sum), downcast='unsigned')
transactions_shops['area_mean_turnover_day'] = pd.to_numeric(transactions_shops.groupby(['area'])['area_day_turnover'].transform(np.mean), downcast='float') 
transactions_shops['area_max_turnover_block'] = pd.to_numeric(transactions_shops_blocks.groupby(['area'])['area_block_turnover'].transform(np.max), downcast='unsigned')
transactions_shops['area_min_turnover_block'] = pd.to_numeric(transactions_shops_blocks.groupby(['area'])['area_block_turnover'].transform(np.min), downcast='unsigned')
transactions_shops['area_max_turnover_day'] = pd.to_numeric(transactions_shops.groupby(['area'])['area_day_turnover'].transform(np.max), downcast='unsigned')
transactions_shops['area_min_turnover_day'] = pd.to_numeric(transactions_shops.groupby(['area'])['area_day_turnover'].transform(np.min), downcast='unsigned')

In [51]:
transactions_shops['area_mean_price'] = pd.to_numeric(transactions_shops.groupby('area')['item_price'].transform(np.mean), downcast='float') 


In [52]:
area_first_two_blocks_units = transactions_shops.groupby(['area','is_first_two_blocks'], as_index=False)['item_cnt_day'].sum()
area_first_two_blocks_units = area_first_two_blocks_units[area_first_two_blocks_units['is_first_two_blocks'] == True].set_index('area').iloc[:,1]
transactions_shops['area_first_two_blocks_units'] = pd.to_numeric(transactions_shops['area'].map(area_first_two_blocks_units), downcast='unsigned')

area_last_two_blocks_units = transactions_shops.groupby(['area','is_last_two_blocks'], as_index=False)['item_cnt_day'].sum()
area_last_two_blocks_units = area_last_two_blocks_units[area_last_two_blocks_units['is_last_two_blocks'] == True].set_index('area').iloc[:,1]
transactions_shops['area_last_two_blocks_units'] = pd.to_numeric(transactions_shops['area'].map(area_last_two_blocks_units), downcast='unsigned')

transactions_shops['area_fluctuation_units_first_last_blocks'] =  pd.to_numeric(((transactions_shops['area_first_two_blocks_units'] - transactions_shops['area_last_two_blocks_units'] ) / \
                                                             transactions_shops['area_first_two_blocks_units']) * 100 * -1, downcast='float') 


area_first_two_blocks_mean_price = transactions_shops.groupby(['area','is_first_two_blocks'], as_index=False)['item_price'].mean()
area_first_two_blocks_mean_price = area_first_two_blocks_mean_price[area_first_two_blocks_mean_price['is_first_two_blocks'] == True].set_index('area').iloc[:,1]
transactions_shops['area_first_two_blocks_mean_price'] = pd.to_numeric(transactions_shops['area'].map(area_first_two_blocks_mean_price), downcast='unsigned')

area_last_two_blocks_mean_price = transactions_shops.groupby(['area','is_last_two_blocks'], as_index=False)['item_price'].mean()
area_last_two_blocks_mean_price = area_last_two_blocks_mean_price[area_last_two_blocks_mean_price['is_last_two_blocks'] == True].set_index('area').iloc[:,1]
transactions_shops['area_last_two_blocks_mean_price'] = pd.to_numeric(transactions_shops['area'].map(area_last_two_blocks_mean_price), downcast='unsigned')

transactions_shops['area_fluctuation_price_first_last_blocks'] =  pd.to_numeric(((transactions_shops['area_first_two_blocks_mean_price'] - transactions_shops['area_last_two_blocks_mean_price'] ) / \
                                                             transactions_shops['area_first_two_blocks_mean_price'])  * 100 * -1, downcast='float') 

In [53]:

transactions_shops.to_pickle("pickled/transactions_shops")
transactions_shops_blocks.to_pickle("pickled/transactions_shops_blocks")


del transactions_shops
del transactions_shops_blocks
gc.collect()

189

shop_category


-UNITS
shop_category_units
shop_category_block_units
shop_category_mean_units_block
shop_category_day_units
shop_category_mean_units_day
shop_category_max_units_block
shop_category_min_units_block
shop_category_max_units_day
shop_category_min_units_day

-TURNOVER
shop_category_turnover
shop_category_block_turnover
shop_category_mean_turnover_block
shop_category_day_turnover
shop_category_mean_turnover_day
shop_category_max_turnover_block
shop_category_min_turnover_block
shop_category_max_turnover_day
shop_category_min_turnover_day

-PRICE
shop_category_mean_price
shop_category_mean_price_block


-TREND
shop_category_first_two_blocks_units
shop_category_last_two_blocks_units
shop_category_fluctuation_units_first_last_blocks
shop_category_first_two_blocks_mean_price
shop_category_last_two_blocks_mean_price
shop_category_fluctuation_price_first_last_blocks

-ENCODINGS
shop_category_share_of_total_units
shop_category_share_of_total_gross

In [54]:
gc.collect()
transactions_shops_categories = transactions.copy()
transactions_shops_categories_blocks = transactions.copy()

In [55]:
transactions_shops_categories_blocks['shop_category_block_units'] = pd.to_numeric(transactions_shops_categories_blocks.groupby(['shop_id','item_category_id','date_block_num'])['item_cnt_day'].transform(np.sum), downcast='unsigned')
transactions_shops_categories_blocks['shop_category_block_turnover'] = pd.to_numeric(transactions_shops_categories_blocks.groupby(['shop_id','item_category_id','date_block_num'])['turnover'].transform(np.sum), downcast='unsigned')
transactions_shops_categories_blocks['shop_category_mean_price_block'] = pd.to_numeric(transactions_shops_categories_blocks.groupby(['shop_id','item_category_id', 'date_block_num'])['item_price'].transform(np.mean), downcast='float') 

In [56]:
transactions_shops_categories['shop_category_units'] = pd.to_numeric(transactions_shops_categories.groupby(['shop_id','item_category_id'])['item_cnt_day'].transform(np.sum), downcast='unsigned')
transactions_shops_categories['shop_category_mean_units_block'] = pd.to_numeric(transactions_shops_categories_blocks.groupby(['shop_id','item_category_id'])['shop_category_block_units'].transform(np.mean), downcast='float') 
transactions_shops_categories['shop_category_day_units'] = pd.to_numeric(transactions_shops_categories.groupby(['shop_id','item_category_id','date'])['item_cnt_day'].transform(np.sum), downcast='unsigned')
transactions_shops_categories['shop_category_mean_units_day'] = pd.to_numeric(transactions_shops_categories.groupby(['shop_id','item_category_id'])['shop_category_day_units'].transform(np.mean), downcast='float') 
transactions_shops_categories['shop_category_max_units_block'] = pd.to_numeric(transactions_shops_categories_blocks.groupby(['shop_id','item_category_id'])['shop_category_block_units'].transform(np.max), downcast='unsigned')
transactions_shops_categories['shop_category_min_units_block'] = pd.to_numeric(transactions_shops_categories_blocks.groupby(['shop_id','item_category_id'])['shop_category_block_units'].transform(np.min), downcast='unsigned')
transactions_shops_categories['shop_category_max_units_day'] = pd.to_numeric(transactions_shops_categories.groupby(['shop_id','item_category_id'])['shop_category_day_units'].transform(np.max), downcast='unsigned')
transactions_shops_categories['shop_category_min_units_day'] = pd.to_numeric(transactions_shops_categories.groupby(['shop_id','item_category_id'])['shop_category_day_units'].transform(np.min), downcast='unsigned')


In [57]:
transactions_shops_categories['shop_category_turnover'] = pd.to_numeric(transactions_shops_categories.groupby(['shop_id','item_category_id'])['turnover'].transform(np.sum), downcast='unsigned')
transactions_shops_categories['shop_category_mean_turnover_block'] = pd.to_numeric(transactions_shops_categories_blocks.groupby(['shop_id','item_category_id'])['shop_category_block_turnover'].transform(np.mean), downcast='float') 
transactions_shops_categories['shop_category_day_turnover'] = pd.to_numeric(transactions_shops_categories.groupby(['shop_id','item_category_id','date'])['turnover'].transform(np.sum), downcast='unsigned')
transactions_shops_categories['shop_category_mean_turnover_day'] = pd.to_numeric(transactions_shops_categories.groupby(['shop_id','item_category_id'])['shop_category_day_turnover'].transform(np.mean), downcast='float') 
transactions_shops_categories['shop_category_max_turnover_block'] = pd.to_numeric(transactions_shops_categories_blocks.groupby(['shop_id','item_category_id'])['shop_category_block_turnover'].transform(np.max), downcast='unsigned')
transactions_shops_categories['shop_category_min_turnover_block'] = pd.to_numeric(transactions_shops_categories_blocks.groupby(['shop_id','item_category_id'])['shop_category_block_turnover'].transform(np.min), downcast='unsigned')
transactions_shops_categories['shop_category_max_turnover_day'] = pd.to_numeric(transactions_shops_categories.groupby(['shop_id','item_category_id'])['shop_category_day_turnover'].transform(np.max), downcast='unsigned')
transactions_shops_categories['shop_category_min_turnover_day'] = pd.to_numeric(transactions_shops_categories.groupby(['shop_id','item_category_id'])['shop_category_day_turnover'].transform(np.min), downcast='unsigned')

In [58]:
transactions_shops_categories['shop_category_mean_price'] = pd.to_numeric(transactions_shops_categories.groupby(['shop_id','item_category_id'])['item_price'].transform(np.mean), downcast='float') 


In [59]:

transactions_shops_categories.to_pickle("pickled/transactions_shops_categories")
transactions_shops_categories_blocks.to_pickle("pickled/transactions_shops_categories_blocks")


del transactions_shops_categories
del transactions_shops_categories_blocks
gc.collect()

245

In [60]:
###
#DEBUG
###


pd.set_option('display.max_columns', None)  
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', -1)
#transactions.sample(10).sort_values(by=['item_units'], ascending=False)
#transactions[transactions['item_category_id'] == 58].sample(10).sort_values(by=['total_sales_units'], ascending=False)

In [61]:
train_item_ids = transactions['item_id'].unique()
train_shop_ids = transactions['shop_id'].unique()
test_item_ids = test['item_id'].unique()
test_shop_ids = test['shop_id'].unique()
train_blocks = transactions['date_block_num'].unique()

all_item_ids = np.unique(np.append(test_item_ids,train_item_ids))
all_shop_ids = np.unique(np.append(train_shop_ids,test_shop_ids))

In [62]:
combinations = []
for shop in all_shop_ids:
    #get all article ids ever associated to this shop
    train_ids = transactions[transactions['shop_id'] == shop]['item_id'].unique()
    test_ids = test[test['shop_id'] == shop]['item_id'].unique()
    all_shop = np.unique(np.append(train_ids, test_ids))
    all_shop_combo = [[item, shop, block] for item in all_shop for block in train_blocks]
    for combo in all_shop_combo:
        combinations.append(combo)

In [63]:
all_combos = pd.DataFrame(np.unique(np.vstack([combinations]), axis=0), columns=['item_id', 'shop_id', 'date_block_num'])

In [64]:
all_combos.head()

,item_id,shop_id,date_block_num
0,0.0,54.0,12.0
1,0.0,54.0,13.0
2,0.0,54.0,14.0
3,0.0,54.0,15.0
4,0.0,54.0,16.0


In [65]:
all_combos['item_id'] = pd.to_numeric(all_combos['item_id'], downcast='unsigned')
all_combos['shop_id'] = pd.to_numeric(all_combos['shop_id'], downcast='unsigned')
all_combos['date_block_num'] = pd.to_numeric(all_combos['date_block_num'], downcast='unsigned')

In [66]:
len(all_combos)

8333930

In [67]:
all_combos = pd.merge(all_combos, items[['item_id', 'item_category_id']], on='item_id', how='left')

In [68]:
dates = transactions[['date_block_num', 'month', 'year']].drop_duplicates(['date_block_num', 'month', 'year'])

dates_dict = {}

for index,row in dates.iterrows():
    dates_dict[row['date_block_num']] = {"month": row['month'], "year": row['year']}
    
dates_dict

{20: {'month': 9, 'year': 2014},
 15: {'month': 4, 'year': 2014},
 18: {'month': 7, 'year': 2014},
 19: {'month': 8, 'year': 2014},
 21: {'month': 10, 'year': 2014},
 22: {'month': 11, 'year': 2014},
 23: {'month': 12, 'year': 2014},
 24: {'month': 1, 'year': 2015},
 27: {'month': 4, 'year': 2015},
 25: {'month': 2, 'year': 2015},
 12: {'month': 1, 'year': 2014},
 14: {'month': 3, 'year': 2014},
 16: {'month': 5, 'year': 2014},
 17: {'month': 6, 'year': 2014},
 13: {'month': 2, 'year': 2014},
 26: {'month': 3, 'year': 2015},
 28: {'month': 5, 'year': 2015},
 29: {'month': 6, 'year': 2015},
 30: {'month': 7, 'year': 2015},
 31: {'month': 8, 'year': 2015},
 32: {'month': 9, 'year': 2015},
 33: {'month': 10, 'year': 2015}}

In [69]:
all_combos['month'] = pd.to_numeric(all_combos['date_block_num'].apply(lambda block: dates_dict[block]['month']), downcast='unsigned')
all_combos['year'] = pd.to_numeric(all_combos['date_block_num'].apply(lambda block: dates_dict[block]['year']), downcast='unsigned')

In [70]:
def downcast(df, columns, dtypes):
    for column in columns:
        dtype = dtypes[column].kind
        if dtype in ['O', 'b']:
            continue  
        if dtype == 'u':
            df[column] = pd.to_numeric(df[column].astype(int), downcast='unsigned')
        elif dtype == 'i':
            df[column] = pd.to_numeric(df[column].astype(int), downcast='signed')
        else:
            df[column] = pd.to_numeric(df[column], downcast='float')

In [71]:
def fillnas(df, columns, dtypes):
    for column in columns:
        dtype = dtypes[column].kind
        if dtype in ['O', 'b']:
            continue  
        df[column].fillna(0, inplace=True)

In [72]:
#del training

In [73]:
transactions_items_columns = ['item_id', 'item_first_block',
       'item_last_block', 'is_first_two_blocks', 'is_last_two_blocks',
       'item_units', 'item_mean_units_block', 'item_day_units',
       'item_mean_units_day', 'item_max_units_block',
       'item_min_units_block', 'item_max_units_day', 'item_min_units_day',
       'item_turnover', 'item_mean_turnover_block', 'item_day_turnover',
       'item_mean_turnover_day', 'item_max_turnover_block',
       'item_min_turnover_block', 'item_max_turnover_day',
       'item_min_turnover_day', 'item_days_of_activity',
       'item_blocks_of_activity', 'item_days_since_start',
       'item_mean_day_between_activity',
       'item_longest_stretch_days_without_activity',
       'item_longest_stretch_blocks_without_activity',
       'item_longest_stretch_block_with_activity',
       'item_number_of_consecutive_days_with_activity',
       'item_days_between_start_and_first_activity',
       'item_blocks_between_start_and_first_activity', 'item_first_day',
       'item_last_day', 'item_activity_on_all_blocks', 'item_mean_price',
       'item_min_price', 'item_max_price', 'item_number_different_prices',
       'item_price_amplitude', 
       'item_deviation_mean_category_price',
       'item_first_two_blocks_units', 'item_last_two_blocks_units',
       'item_fluctuation_units_first_last_blocks',
       'item_first_two_blocks_mean_price',
       'item_last_two_blocks_mean_price',
       'item_fluctuation_price_first_last_blocks',
       'item_share_of_total_units', 'item_share_of_total_turnover']

In [74]:
transactions_items = pd.read_pickle("pickled/transactions_items")
transactions_items_dtypes = transactions_items.dtypes
training = pd.merge(all_combos, transactions_items[transactions_items_columns].drop_duplicates('item_id'), on=['item_id'], how='left', copy=False)

del transactions_items
fillnas(training, transactions_items_columns, transactions_items_dtypes)
downcast(training, transactions_items_columns, transactions_items_dtypes)
gc.collect()

42

In [75]:
transactions_items_blocks_columns =  [ 'date_block_num', 'item_id', 'item_block_units',
 'item_block_turnover',
 'item_mean_price_block']

In [76]:
transactions_items_blocks = pd.read_pickle("pickled/transactions_items_blocks")
transactions_items_blocks_dtypes = transactions_items_blocks.dtypes

training = pd.merge(training, transactions_items_blocks[transactions_items_blocks_columns]\
                    .drop_duplicates(['item_id', 'date_block_num']), on=['item_id','date_block_num'], how='left', copy=False)

del transactions_items_blocks
training.fillna(0, inplace=True)
downcast(training, transactions_items_blocks_columns, transactions_items_blocks_dtypes)
gc.collect()

14

In [77]:
gc.collect()
training.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8333930 entries, 0 to 8333929
Data columns (total 56 columns):
item_id                                          int16
shop_id                                          uint8
date_block_num                                   uint8
item_category_id                                 uint8
month                                            uint8
year                                             uint16
item_first_block                                 uint8
item_last_block                                  uint8
is_first_two_blocks                              object
is_last_two_blocks                               object
item_units                                       float32
item_mean_units_block                            float32
item_day_units                                   int16
item_mean_units_day                              float32
item_max_units_block                             int16
item_min_units_block                             int1

In [78]:
transactions_categories_columns = [
       'item_category_id',
       'category_units', 'category_mean_units_block',
       'category_day_units', 'category_mean_units_day',
       'category_max_units_block', 'category_min_units_block',
       'category_max_units_day', 'category_min_units_day',
       'category_turnover', 'category_mean_turnover_block',
       'category_day_turnover', 'category_mean_turnover_day',
       'category_max_turnover_block', 'category_min_turnover_block',
       'category_max_turnover_day', 'category_min_turnover_day',
       'category_mean_price', 'category_min_price', 'category_max_price',
       'category_first_two_blocks_units',
       'category_last_two_blocks_units',
       'category_fluctuation_units_first_last_blocks',
       'category_first_two_blocks_mean_price',
       'category_last_two_blocks_mean_price',
       'category_fluctuation_price_first_last_blocks', 'subcategory',
       'video_game', 'gaming_old_gen', 'gaming_new_gen', 'pc_games',
       'payment_cards', 'movies', 'movies_niche', 'books', 'music',
       'music_CD', 'music_vinyl', 'gifts', 'software',
       'subcategory_units',
       'subcategory_mean_units_block', 'subcategory_day_units',
       'subcategory_mean_units_day', 'subcategory_max_units_block',
       'subcategory_min_units_block', 'subcategory_max_units_day',
       'subcategory_min_units_day', 'subcategory_turnover', 'subcategory_mean_turnover_block',
       'subcategory_day_turnover', 'subcategory_mean_turnover_day',
       'subcategory_max_turnover_block', 'subcategory_min_turnover_block',
       'subcategory_max_turnover_day', 'subcategory_min_turnover_day',
       'category_share_of_total_units',
       'category_share_of_total_turnover',
       'subcategory_share_of_total_units',
       'subcategory_share_of_total_turnover',
       'subcategory_first_two_blocks_units',
       'subcategory_last_two_blocks_units',
       'subcategory_fluctuation_units_first_last_blocks',
       'subcategory_first_two_blocks_mean_price',
       'subcategory_last_two_blocks_mean_price',
       'subcategory_fluctuation_price_first_last_blocks']


In [79]:
training.columns.values

array(['item_id', 'shop_id', 'date_block_num', 'item_category_id',
       'month', 'year', 'item_first_block', 'item_last_block',
       'is_first_two_blocks', 'is_last_two_blocks', 'item_units',
       'item_mean_units_block', 'item_day_units', 'item_mean_units_day',
       'item_max_units_block', 'item_min_units_block',
       'item_max_units_day', 'item_min_units_day', 'item_turnover',
       'item_mean_turnover_block', 'item_day_turnover',
       'item_mean_turnover_day', 'item_max_turnover_block',
       'item_min_turnover_block', 'item_max_turnover_day',
       'item_min_turnover_day', 'item_days_of_activity',
       'item_blocks_of_activity', 'item_days_since_start',
       'item_mean_day_between_activity',
       'item_longest_stretch_days_without_activity',
       'item_longest_stretch_blocks_without_activity',
       'item_longest_stretch_block_with_activity',
       'item_number_of_consecutive_days_with_activity',
       'item_days_between_start_and_first_activity',
       '

In [80]:
transactions_categories = pd.read_pickle("pickled/transactions_categories")
transactions_categories_dtypes = transactions_categories.dtypes
training = pd.merge(training, transactions_categories[transactions_categories_columns]\
                    .drop_duplicates('item_category_id'), on=['item_category_id'], how='left', copy=False)

del transactions_categories
fillnas(training, transactions_categories_columns, transactions_categories_dtypes)
downcast(training, transactions_categories_columns, transactions_categories_dtypes)
gc.collect()

21

In [81]:
transactions_categories_blocks_columns = ['item_category_id', 'date_block_num', 'category_block_units',
 'category_block_turnover',
 'category_mean_price_block', 'subcategory_block_units',
 'subcategory_block_turnover',
 'subcategory_mean_price_block']

In [82]:
transactions_categories_blocks = pd.read_pickle("pickled/transactions_categories_blocks")
transactions_categories_blocks_dtypes = transactions_categories_blocks.dtypes
training = pd.merge(training, transactions_categories_blocks[transactions_categories_blocks_columns]\
                    .drop_duplicates(['item_category_id', 'date_block_num']), on=['item_category_id', 'date_block_num'], how='left', copy=False)

del transactions_categories_blocks
fillnas(training, transactions_categories_blocks_columns, transactions_categories_blocks_dtypes)
downcast(training, transactions_categories_blocks_columns, transactions_categories_blocks_dtypes)
gc.collect()

14

In [83]:
transactions_shops_columns = ['shop_id', 
       'shop_units', 'shop_mean_units_block', 'shop_day_units',
       'shop_mean_units_day', 'shop_max_units_block',
       'shop_min_units_block', 'shop_max_units_day', 'shop_min_units_day',
       'shop_turnover', 'shop_mean_turnover_block', 'shop_day_turnover',
       'shop_mean_turnover_day', 'shop_max_turnover_block',
       'shop_min_turnover_block', 'shop_max_turnover_day',
       'shop_min_turnover_day', 'shop_mean_price',
       'shop_first_two_blocks_units', 'shop_last_two_blocks_units',
       'shop_fluctuation_units_first_last_blocks',
       'shop_first_two_blocks_mean_price',
       'shop_last_two_blocks_mean_price',
       'shop_fluctuation_price_first_last_blocks', 'shop_share_of_units',
       'shop_share_of_turnover', 'shop_TC', 'shop_TRK', 'shop_SEC',
       'shop_shopping_center', 'shop_moscow', 'max_category_units',
       'max_category_turnover', 'area', 'area_units',
       'area_mean_units_block', 'area_day_units', 'area_mean_units_day',
       'area_max_units_block', 'area_min_units_block',
       'area_max_units_day', 'area_min_units_day', 'area_turnover',
     'area_mean_turnover_block',
       'area_day_turnover', 'area_mean_turnover_day',
       'area_max_turnover_block', 'area_min_turnover_block',
       'area_max_turnover_day', 'area_min_turnover_day',
       'area_mean_price',
       'area_first_two_blocks_units', 'area_last_two_blocks_units',
       'area_fluctuation_units_first_last_blocks',
       'area_first_two_blocks_mean_price',
       'area_last_two_blocks_mean_price',
       'area_fluctuation_price_first_last_blocks']

In [84]:
transactions_shops = pd.read_pickle("pickled/transactions_shops")
transactions_shops_dtypes = transactions_shops.dtypes
training = pd.merge(training, transactions_shops[transactions_shops_columns]\
                    .drop_duplicates('shop_id'), on=['shop_id'], how='left', copy=False)

del transactions_shops
fillnas(training, transactions_shops_columns, transactions_shops_dtypes)
downcast(training, transactions_shops_columns, transactions_shops_dtypes)
gc.collect()

14

In [85]:
transactions_shops_blocks_columns = ['shop_id', 'date_block_num',  'shop_block_units',
 'shop_block_turnover',
 'shop_mean_price_block', 'area_block_units',
 'area_block_turnover',
 'area_mean_price_block']

In [86]:
transactions_shops_blocks = pd.read_pickle("pickled/transactions_shops_blocks")
transactions_shops_blocks_dtypes = transactions_shops_blocks.dtypes
training = pd.merge(training, transactions_shops_blocks[transactions_shops_blocks_columns]\
                    .drop_duplicates(['shop_id', 'date_block_num']), on=['shop_id', 'date_block_num'], how='left', copy=False)

del transactions_shops_blocks
fillnas(training, transactions_shops_blocks_columns, transactions_shops_blocks_dtypes)
downcast(training, transactions_shops_blocks_columns, transactions_shops_blocks_dtypes)
gc.collect()

14

In [87]:
transactions_shops_categories_columns = [ 'shop_id', 
       'item_category_id',
       'shop_category_units', 'shop_category_mean_units_block',
       'shop_category_day_units', 'shop_category_mean_units_day',
       'shop_category_max_units_block', 'shop_category_min_units_block',
       'shop_category_max_units_day', 'shop_category_min_units_day',
       'shop_category_turnover', 'shop_category_mean_turnover_block',
       'shop_category_day_turnover', 'shop_category_mean_turnover_day',
       'shop_category_max_turnover_block',
       'shop_category_min_turnover_block',
       'shop_category_max_turnover_day', 'shop_category_min_turnover_day',
       'shop_category_mean_price']

In [88]:
transactions_shops_categories = pd.read_pickle("pickled/transactions_shops_categories")
transactions_shops_categories_dtypes = transactions_shops_categories.dtypes
training = pd.merge(training, transactions_shops_categories[transactions_shops_categories_columns]\
                    .drop_duplicates(['shop_id','item_category_id']), on=['shop_id','item_category_id'], how='left', copy=False)

del transactions_shops_categories
fillnas(training, transactions_shops_categories_columns, transactions_shops_categories_dtypes)
downcast(training, transactions_shops_categories_columns, transactions_shops_categories_dtypes)
gc.collect()

14

In [89]:
transactions_shops_categories_blocks_columns = ['shop_id', 'item_category_id', 'date_block_num',   'shop_category_block_units',
 'shop_category_block_turnover',
 'shop_category_mean_price_block']

In [90]:
transactions_shops_categories_blocks = pd.read_pickle("pickled/transactions_shops_categories_blocks")
transactions_shops_categories_blocks_dtypes = transactions_shops_categories_blocks.dtypes
training = pd.merge(training, transactions_shops_categories_blocks[transactions_shops_categories_blocks_columns]\
                    .drop_duplicates(['shop_id','item_category_id','date_block_num']),\
                    on=['shop_id','item_category_id','date_block_num'], how='left', copy=False)

del transactions_shops_categories_blocks
fillnas(training, transactions_shops_categories_blocks_columns, transactions_shops_categories_blocks_dtypes)
downcast(training, transactions_shops_categories_blocks_columns, transactions_shops_categories_blocks_dtypes)
gc.collect()

14

In [91]:
len(training)

8333930

In [118]:

#training.to_pickle("pickled/training_pre_lags")
training = pd.read_pickle("pickled/training_pre_lags")

In [119]:
lag_columns = [
 'item_block_units',
 'item_block_turnover',
 'item_mean_price_block',
 'category_block_units',
 'category_block_turnover',
 'category_mean_price_block',
 'subcategory_block_units',
 'subcategory_block_turnover',
 'subcategory_mean_price_block',
 'shop_block_units',
 'shop_block_turnover',
 'shop_mean_price_block',
 'area_block_units',
 'area_block_turnover',
 'area_mean_price_block',
 'shop_category_block_units',
 'shop_category_turnover',
 'shop_category_mean_price_block'
 ]

In [8]:
def downcast_lags(df, lagged_names):
    for lagged_name in lagged_names:
        df[lagged_name].fillna(0,inplace=True)    
    for column in lagged_names:
        if "mean" in column:
            df[column] = pd.to_numeric(df[column], downcast='float')
        else:
            df[column] = pd.to_numeric(df[column].astype(int), downcast='unsigned')
    return df

In [9]:
lags = [1,2,3]


def add_lag_features(df, lag_columns, idx_columns):

    gc.collect()
    def lagged_name(lag_column, lag):
        return "%s_lag_%d" % (lag_column, lag)

    merge_columns = ['lagged_block'] + idx_columns

    for lag in lags:
        print(lag)
        lagged = df[['date_block_num'] + idx_columns + lag_columns].copy()
        lagged.rename(columns={'date_block_num':'lagged_block'},inplace=True)
        df['lagged_block'] = df['date_block_num'] - lag
        lagged_names = [lagged_name(c,lag) for c in lag_columns]
        lag_mapping = dict(zip(lag_columns, lagged_names))
        lagged.rename(columns=lag_mapping,inplace=True)
        df = pd.merge(df.set_index(merge_columns),lagged[lagged_names+merge_columns]\
                        .drop_duplicates(lagged_names+merge_columns)\
                        .set_index(merge_columns)\
                        ,on=merge_columns,how='left',copy=False)
        gc.collect()
        df.reset_index(inplace=True)
    
        df = downcast_lags(df, lagged_names)
        del lagged
        gc.collect()
        
    return df

In [122]:
gc.collect()
def lagged_name(lag_column, lag):
    return "%s_lag_%d" % (lag_column, lag)

lag_columns = [
 'item_block_units',
 'item_block_turnover',
 'item_mean_price_block',
 ]

idx_columns = ['item_id']

training = add_lag_features(training,lag_columns,idx_columns)

1
2
3


In [127]:
training.columns.values

array(['lagged_block', 'item_category_id', 'item_id', 'shop_id',
       'date_block_num', 'month', 'year', 'item_first_block',
       'item_last_block', 'is_first_two_blocks', 'is_last_two_blocks',
       'item_units', 'item_mean_units_block', 'item_day_units',
       'item_mean_units_day', 'item_max_units_block',
       'item_min_units_block', 'item_max_units_day', 'item_min_units_day',
       'item_turnover', 'item_mean_turnover_block', 'item_day_turnover',
       'item_mean_turnover_day', 'item_max_turnover_block',
       'item_min_turnover_block', 'item_max_turnover_day',
       'item_min_turnover_day', 'item_days_of_activity',
       'item_blocks_of_activity', 'item_days_since_start',
       'item_mean_day_between_activity',
       'item_longest_stretch_days_without_activity',
       'item_longest_stretch_blocks_without_activity',
       'item_longest_stretch_block_with_activity',
       'item_number_of_consecutive_days_with_activity',
       'item_days_between_start_and_first_act

In [124]:

pd.set_option('display.max_columns', None)  
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', -1)
training[(training['item_id'] == 30) & (training['shop_id'] == 30)]\
                .drop_duplicates(['item_id', 'date_block_num'])[['item_id','shop_id','date_block_num','item_block_units','item_block_turnover',\
                'item_block_units_lag_1',
 'item_block_turnover_lag_1', 'item_mean_price_block_lag_1',
 'item_block_units_lag_2', 'item_block_turnover_lag_2',
 'item_mean_price_block_lag_2', 'item_block_units_lag_3',
 'item_block_turnover_lag_3' ,'item_mean_price_block_lag_3']]

,item_id,shop_id,date_block_num,item_block_units,item_block_turnover,item_block_units_lag_1,item_block_turnover_lag_1,item_mean_price_block_lag_1,item_block_units_lag_2,item_block_turnover_lag_2,item_mean_price_block_lag_2,item_block_units_lag_3,item_block_turnover_lag_3,item_mean_price_block_lag_3
1210,30,30,12,58,9802,0,0,0.000000,0,0,0.000000,0,0,0.000000
1211,30,30,13,24,3986,58,9802,169.000000,0,0,0.000000,0,0,0.000000
1212,30,30,14,31,5239,24,3986,166.083328,58,9802,169.000000,0,0,0.000000
1213,30,30,15,21,3479,31,5239,169.000000,24,3986,166.083328,58,9802,169.000000
1214,30,30,16,16,2634,21,3479,165.666672,31,5239,169.000000,24,3986,166.083328
1215,30,30,17,13,2197,16,2634,164.625000,21,3479,165.666672,31,5239,169.000000
1216,30,30,18,13,2127,13,2197,169.000000,16,2634,164.625000,21,3479,165.666672
1217,30,30,19,12,2028,13,2127,163.615387,13,2197,169.000000,16,2634,164.625000
1218,30,30,20,11,1859,12,2028,169.000000,13,2127,163.615387,13,2197,169.000000
1219,30,30,21,13,2197,11,1859,169.000000,12,2028,169.000000,13,2127,163.615387


In [125]:
gc.collect()
def lagged_name(lag_column, lag):
    return "%s_lag_%d" % (lag_column, lag)

lag_columns = [
 'category_block_units',
 'category_block_turnover',
 'category_mean_price_block',
 'subcategory_block_units',
 'subcategory_block_turnover',
 'subcategory_mean_price_block',
 ]

idx_columns = ['item_category_id']

training = add_lag_features(training,lag_columns,idx_columns)

1
2
3


In [10]:
gc.collect()
#training.to_pickle("pickled/training_mid_lags")
training = pd.read_pickle("pickled/training_mid_lags")

In [11]:
gc.collect()
def lagged_name(lag_column, lag):
    return "%s_lag_%d" % (lag_column, lag)

lag_columns = [
 'shop_block_units',
 'shop_block_turnover',
 'shop_mean_price_block',
 'area_block_units',
 'area_block_turnover',
 'area_mean_price_block'
 ]

idx_columns = ['shop_id']


training = add_lag_features(training,lag_columns,idx_columns)

1
2
3


In [12]:
lag_columns = [
  'shop_category_block_units',
 'shop_category_turnover',
 'shop_category_mean_price_block'
 ]

idx_columns = ['shop_id','item_category_id']


training = add_lag_features(training,lag_columns,idx_columns)

1
2
3


In [13]:
training.drop(columns=['lagged_block'],inplace=True)

In [14]:
training.columns.values

array(['shop_id', 'item_category_id', 'item_id', 'date_block_num',
       'month', 'year', 'item_first_block', 'item_last_block',
       'is_first_two_blocks', 'is_last_two_blocks', 'item_units',
       'item_mean_units_block', 'item_day_units', 'item_mean_units_day',
       'item_max_units_block', 'item_min_units_block',
       'item_max_units_day', 'item_min_units_day', 'item_turnover',
       'item_mean_turnover_block', 'item_day_turnover',
       'item_mean_turnover_day', 'item_max_turnover_block',
       'item_min_turnover_block', 'item_max_turnover_day',
       'item_min_turnover_day', 'item_days_of_activity',
       'item_blocks_of_activity', 'item_days_since_start',
       'item_mean_day_between_activity',
       'item_longest_stretch_days_without_activity',
       'item_longest_stretch_blocks_without_activity',
       'item_longest_stretch_block_with_activity',
       'item_number_of_consecutive_days_with_activity',
       'item_days_between_start_and_first_activity',
       '

In [16]:
gc.collect()
training.to_pickle("pickled/training_post_lags")

In [17]:
gc.collect()
training.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8333930 entries, 0 to 8333929
Columns: 263 entries, shop_id to shop_category_mean_price_block_lag_3
dtypes: bool(5), category(2), float32(96), int16(22), int32(53), int64(1), int8(2), object(16), uint16(31), uint32(23), uint64(2), uint8(10)
memory usage: 10.5 GB


In [18]:
training.dtypes

shop_id                                  uint64
item_category_id                         uint64
item_id                                   int64
date_block_num                            uint8
month                                     uint8
year                                     uint16
item_first_block                          uint8
item_last_block                           uint8
is_first_two_blocks                      object
is_last_two_blocks                       object
item_units                              float32
item_mean_units_block                   float32
item_day_units                            int16
item_mean_units_day                     float32
item_max_units_block                      int16
item_min_units_block                      int16
item_max_units_day                        int16
item_min_units_day                         int8
item_turnover                             int32
item_mean_turnover_block                float32
item_day_turnover                       

In [19]:
cols = ['shop_id','item_id', 'date_block_num']
training = pd.merge(training.set_index(cols), transactions[cols+['y']].drop_duplicates(cols).set_index(cols), on=cols, how='left', copy=False)

transactions.reset_index(inplace=True)
training.reset_index(inplace=True)

In [20]:
training['y'] = training['y'].fillna(0)

In [3]:
#training.to_pickle("pickled/training_pre_catboost")
training = pd.read_pickle("pickled/training")

In [23]:
gc.collect()

189

In [34]:
pd.set_option('display.max_rows', 300)
for col in training.columns:
    if training.dtypes[col].kind == 'b':
        print(col, training.dtypes[col])

shop_TC bool
shop_TRK bool
shop_SEC bool
shop_shopping_center bool
shop_moscow bool
1 bool
2 bool
3 bool
4 bool
5 bool
6 bool
7 bool
8 bool
9 bool
10 bool
11 bool
12 bool


In [ ]:
for m in range(1,13):
    training[str(m)] = training['month'] == m

In [5]:
x_train = training[training['date_block_num'] < 33]
y_train = x_train['y']
#x_train = x_train.drop(columns=['y'])

x_val = training[training['date_block_num'] == 33]
y_val = x_val['y']
#x_val = x_val.drop(columns=['y'])


In [6]:
del training
gc.collect()

95

In [7]:
x_train.head()

,shop_id,item_id,date_block_num,item_category_id,month,year,item_first_block,item_last_block,is_first_two_blocks,is_last_two_blocks,...,3,4,5,6,7,8,9,10,11,12
0,54,0,12,40,1,2014,20,20,0.0,1.0,...,False,False,False,False,False,False,False,False,False,False
1,54,0,13,40,2,2014,20,20,0.0,1.0,...,False,False,False,False,False,False,False,False,False,False
2,54,0,14,40,3,2014,20,20,0.0,1.0,...,True,False,False,False,False,False,False,False,False,False
3,54,0,15,40,4,2014,20,20,0.0,1.0,...,False,True,False,False,False,False,False,False,False,False
4,54,0,16,40,5,2014,20,20,0.0,1.0,...,False,False,True,False,False,False,False,False,False,False


In [36]:
training.columns.values

array(['shop_id', 'item_id', 'date_block_num', 'item_category_id',
       'month', 'year', 'item_first_block', 'item_last_block',
       'is_first_two_blocks', 'is_last_two_blocks', 'item_units',
       'item_mean_units_block', 'item_day_units', 'item_mean_units_day',
       'item_max_units_block', 'item_min_units_block',
       'item_max_units_day', 'item_min_units_day', 'item_turnover',
       'item_mean_turnover_block', 'item_day_turnover',
       'item_mean_turnover_day', 'item_max_turnover_block',
       'item_min_turnover_block', 'item_max_turnover_day',
       'item_min_turnover_day', 'item_days_of_activity',
       'item_blocks_of_activity', 'item_days_since_start',
       'item_mean_day_between_activity',
       'item_longest_stretch_days_without_activity',
       'item_longest_stretch_blocks_without_activity',
       'item_longest_stretch_block_with_activity',
       'item_number_of_consecutive_days_with_activity',
       'item_days_between_start_and_first_activity',
       '

In [9]:
cb_features = [
       'item_first_block', 'item_last_block',
       'is_first_two_blocks', 'is_last_two_blocks', 'item_units',
       'item_mean_units_block', 'item_mean_units_day',
       'item_max_units_block', 'item_min_units_block',
       'item_max_units_day', 'item_min_units_day', 'item_turnover',
       'item_mean_turnover_block',
       'item_mean_turnover_day', 'item_max_turnover_block',
       'item_min_turnover_block', 'item_max_turnover_day',
       'item_min_turnover_day', 'item_days_of_activity',
       'item_blocks_of_activity', 'item_days_since_start',
       'item_mean_day_between_activity',
       'item_longest_stretch_days_without_activity',
       'item_longest_stretch_blocks_without_activity',
       'item_longest_stretch_block_with_activity',
       'item_number_of_consecutive_days_with_activity',
       'item_days_between_start_and_first_activity',
       'item_blocks_between_start_and_first_activity', 'item_first_day',
       'item_last_day', 'item_activity_on_all_blocks', 'item_mean_price',
       'item_min_price', 'item_max_price', 'item_number_different_prices',
       'item_price_amplitude', 'item_deviation_mean_category_price',
       'item_first_two_blocks_units', 'item_last_two_blocks_units',
       'item_fluctuation_units_first_last_blocks',
       'item_first_two_blocks_mean_price',
       'item_last_two_blocks_mean_price',
       'item_fluctuation_price_first_last_blocks',
       'item_share_of_total_units', 'item_share_of_total_turnover',
       'category_units', 'category_mean_units_block',
       'category_mean_units_day',
       'category_max_units_block', 'category_min_units_block',
       'category_max_units_day', 'category_min_units_day',
       'category_turnover', 'category_mean_turnover_block',
       'category_mean_turnover_day',
       'category_max_turnover_block', 'category_min_turnover_block',
       'category_max_turnover_day', 'category_min_turnover_day',
       'category_mean_price', 'category_min_price', 'category_max_price',
       'category_first_two_blocks_units',
       'category_last_two_blocks_units',
       'category_fluctuation_units_first_last_blocks',
       'category_first_two_blocks_mean_price',
       'category_last_two_blocks_mean_price',
       'category_fluctuation_price_first_last_blocks',
       'video_game', 'gaming_old_gen', 'gaming_new_gen', 'pc_games',
       'payment_cards', 'movies', 'movies_niche', 'books', 'music',
       'music_CD', 'music_vinyl', 'gifts', 'software',
       'subcategory_units', 'subcategory_mean_units_block',
       'subcategory_mean_units_day',
       'subcategory_max_units_block', 'subcategory_min_units_block',
       'subcategory_max_units_day', 'subcategory_min_units_day',
       'subcategory_turnover', 'subcategory_mean_turnover_block',
       'subcategory_mean_turnover_day',
       'subcategory_max_turnover_block', 'subcategory_min_turnover_block',
       'subcategory_max_turnover_day', 'subcategory_min_turnover_day',
       'category_share_of_total_units',
       'category_share_of_total_turnover',
       'subcategory_share_of_total_units',
       'subcategory_share_of_total_turnover',
       'subcategory_first_two_blocks_units',
       'subcategory_last_two_blocks_units',
       'subcategory_fluctuation_units_first_last_blocks',
       'subcategory_first_two_blocks_mean_price',
       'subcategory_last_two_blocks_mean_price',
       'subcategory_fluctuation_price_first_last_blocks',
       'shop_units', 'shop_mean_units_block', 
       'shop_mean_units_day', 'shop_max_units_block',
       'shop_min_units_block', 'shop_max_units_day', 'shop_min_units_day',
       'shop_turnover', 'shop_mean_turnover_block', 
       'shop_mean_turnover_day', 'shop_max_turnover_block',
       'shop_min_turnover_block', 'shop_max_turnover_day',
       'shop_min_turnover_day', 'shop_mean_price',
       'shop_first_two_blocks_units', 'shop_last_two_blocks_units',
       'shop_fluctuation_units_first_last_blocks',
       'shop_first_two_blocks_mean_price',
       'shop_last_two_blocks_mean_price',
       'shop_fluctuation_price_first_last_blocks', 'shop_share_of_units',
       'shop_share_of_turnover', 'shop_TC', 'shop_TRK', 'shop_SEC',
       'shop_shopping_center', 'shop_moscow', 'max_category_units',
       'max_category_turnover', 'area_units',
       'area_mean_units_block', 'area_mean_units_day',
       'area_max_units_block', 'area_min_units_block',
       'area_max_units_day', 'area_min_units_day', 'area_turnover',
       'area_mean_turnover_block',
       'area_mean_turnover_day', 'area_max_turnover_block',
       'area_min_turnover_block', 'area_max_turnover_day',
       'area_min_turnover_day', 'area_mean_price',
       'area_first_two_blocks_units', 'area_last_two_blocks_units',
       'area_fluctuation_units_first_last_blocks',
       'area_first_two_blocks_mean_price',
       'area_last_two_blocks_mean_price',
       'area_fluctuation_price_first_last_blocks',
       'shop_category_units', 'shop_category_mean_units_block',
       'shop_category_mean_units_day',
       'shop_category_max_units_block', 'shop_category_min_units_block',
       'shop_category_max_units_day', 'shop_category_min_units_day',
       'shop_category_turnover', 'shop_category_mean_turnover_block',
       'shop_category_mean_turnover_day',
       'shop_category_max_turnover_block',
       'shop_category_min_turnover_block',
       'shop_category_max_turnover_day', 'shop_category_min_turnover_day',
       'shop_category_mean_price',
       'item_block_units_lag_1', 'item_block_turnover_lag_1',
       'item_mean_price_block_lag_1', 'item_block_units_lag_2',
       'item_block_turnover_lag_2', 'item_mean_price_block_lag_2',
       'item_block_units_lag_3', 'item_block_turnover_lag_3',
       'item_mean_price_block_lag_3', 'category_block_units_lag_1',
       'category_block_turnover_lag_1', 'category_mean_price_block_lag_1',
       'subcategory_block_units_lag_1',
       'subcategory_block_turnover_lag_1',
       'subcategory_mean_price_block_lag_1', 'category_block_units_lag_2',
       'category_block_turnover_lag_2', 'category_mean_price_block_lag_2',
       'subcategory_block_units_lag_2',
       'subcategory_block_turnover_lag_2',
       'subcategory_mean_price_block_lag_2', 'category_block_units_lag_3',
       'category_block_turnover_lag_3', 'category_mean_price_block_lag_3',
       'subcategory_block_units_lag_3',
       'subcategory_block_turnover_lag_3',
       'subcategory_mean_price_block_lag_3', 'shop_block_units_lag_1',
       'shop_block_turnover_lag_1', 'shop_mean_price_block_lag_1',
       'area_block_units_lag_1', 'area_block_turnover_lag_1',
       'area_mean_price_block_lag_1', 'shop_block_units_lag_2',
       'shop_block_turnover_lag_2', 'shop_mean_price_block_lag_2',
       'area_block_units_lag_2', 'area_block_turnover_lag_2',
       'area_mean_price_block_lag_2', 'shop_block_units_lag_3',
       'shop_block_turnover_lag_3', 'shop_mean_price_block_lag_3',
       'area_block_units_lag_3', 'area_block_turnover_lag_3',
       'area_mean_price_block_lag_3',
       '1', '2', '3', '4', '5', '6','7', '8', '9', '10', '11', '12']


In [35]:
cb_model = CatBoostRegressor(iterations=70000,
                             learning_rate=0.001,
                             eval_metric='RMSE',
                             #thread_count=16,
                             task_type = "GPU",
                             use_best_model=True,
                             #l2_leaf_reg = 1000,
                             od_type = "Iter",
                             od_wait = 30,
                             #random_strength = 10,
                             #bagging_temperature = 1,
                             #one_hot_max_size = 2,
                             random_seed = 42)

#drops = ['subcategory','area']
#x_train = x_train.drop(columns=drops)
#x_val = x_val.drop(columns=drops)


cb_model.fit(x_train[cb_features], y_train, #cat_features=categorical_features_indices,
             eval_set=(x_val[cb_features],y_val),
             #cat_features=categorical_features_pos,         
             verbose=True)

0:	learn: 1.0781136	test: 0.9237295	best: 0.9237295 (0)	total: 112ms	remaining: 2h 11m 6s
1:	learn: 1.0777911	test: 0.9235398	best: 0.9235398 (1)	total: 210ms	remaining: 2h 2m 27s
2:	learn: 1.0774677	test: 0.9233501	best: 0.9233501 (2)	total: 305ms	remaining: 1h 58m 45s
3:	learn: 1.0771454	test: 0.9231610	best: 0.9231610 (3)	total: 396ms	remaining: 1h 55m 32s
4:	learn: 1.0768224	test: 0.9229725	best: 0.9229725 (4)	total: 488ms	remaining: 1h 53m 50s
5:	learn: 1.0765008	test: 0.9227837	best: 0.9227837 (5)	total: 580ms	remaining: 1h 52m 44s
6:	learn: 1.0761802	test: 0.9225964	best: 0.9225964 (6)	total: 671ms	remaining: 1h 51m 53s
7:	learn: 1.0758589	test: 0.9224089	best: 0.9224089 (7)	total: 763ms	remaining: 1h 51m 15s
8:	learn: 1.0755391	test: 0.9222216	best: 0.9222216 (8)	total: 855ms	remaining: 1h 50m 48s
9:	learn: 1.0752208	test: 0.9220348	best: 0.9220348 (9)	total: 946ms	remaining: 1h 50m 18s
10:	learn: 1.0749020	test: 0.9218485	best: 0.9218485 (10)	total: 1.04s	remaining: 1h 49m 54s

90:	learn: 1.0508816	test: 0.9083246	best: 0.9083246 (90)	total: 8.28s	remaining: 1h 45m 57s
91:	learn: 1.0505935	test: 0.9081286	best: 0.9081286 (91)	total: 8.37s	remaining: 1h 45m 57s
92:	learn: 1.0503102	test: 0.9079697	best: 0.9079697 (92)	total: 8.46s	remaining: 1h 45m 58s
93:	learn: 1.0500240	test: 0.9077756	best: 0.9077756 (93)	total: 8.55s	remaining: 1h 45m 57s
94:	learn: 1.0497431	test: 0.9076400	best: 0.9076400 (94)	total: 8.64s	remaining: 1h 45m 57s
95:	learn: 1.0494643	test: 0.9074814	best: 0.9074814 (95)	total: 8.73s	remaining: 1h 45m 56s
96:	learn: 1.0491837	test: 0.9073466	best: 0.9073466 (96)	total: 8.82s	remaining: 1h 45m 56s
97:	learn: 1.0488991	test: 0.9071539	best: 0.9071539 (97)	total: 8.91s	remaining: 1h 45m 56s
98:	learn: 1.0486140	test: 0.9069603	best: 0.9069603 (98)	total: 9s	remaining: 1h 45m 57s
99:	learn: 1.0483352	test: 0.9068256	best: 0.9068256 (99)	total: 9.09s	remaining: 1h 45m 57s
100:	learn: 1.0480523	test: 0.9066334	best: 0.9066334 (100)	total: 9.18s	

177:	learn: 1.0275876	test: 0.8947875	best: 0.8947875 (177)	total: 16.1s	remaining: 1h 45m 24s
178:	learn: 1.0273351	test: 0.8946192	best: 0.8946192 (178)	total: 16.2s	remaining: 1h 45m 24s
179:	learn: 1.0270864	test: 0.8945028	best: 0.8945028 (179)	total: 16.3s	remaining: 1h 45m 24s
180:	learn: 1.0268328	test: 0.8943591	best: 0.8943591 (180)	total: 16.4s	remaining: 1h 45m 23s
181:	learn: 1.0265819	test: 0.8942240	best: 0.8942240 (181)	total: 16.5s	remaining: 1h 45m 23s
182:	learn: 1.0263319	test: 0.8940597	best: 0.8940597 (182)	total: 16.6s	remaining: 1h 45m 22s
183:	learn: 1.0260782	test: 0.8939177	best: 0.8939177 (183)	total: 16.7s	remaining: 1h 45m 20s
184:	learn: 1.0258298	test: 0.8937518	best: 0.8937518 (184)	total: 16.8s	remaining: 1h 45m 25s
185:	learn: 1.0255792	test: 0.8936173	best: 0.8936173 (185)	total: 16.9s	remaining: 1h 45m 25s
186:	learn: 1.0253344	test: 0.8935020	best: 0.8935020 (186)	total: 16.9s	remaining: 1h 45m 25s
187:	learn: 1.0250835	test: 0.8933331	best: 0.8933

265:	learn: 1.0067841	test: 0.8833964	best: 0.8833964 (265)	total: 24.2s	remaining: 1h 45m 37s
266:	learn: 1.0065607	test: 0.8832761	best: 0.8832761 (266)	total: 24.3s	remaining: 1h 45m 36s
267:	learn: 1.0063416	test: 0.8831749	best: 0.8831749 (267)	total: 24.4s	remaining: 1h 45m 36s
268:	learn: 1.0061245	test: 0.8830607	best: 0.8830607 (268)	total: 24.4s	remaining: 1h 45m 36s
269:	learn: 1.0059052	test: 0.8829239	best: 0.8829239 (269)	total: 24.5s	remaining: 1h 45m 35s
270:	learn: 1.0056820	test: 0.8828049	best: 0.8828049 (270)	total: 24.6s	remaining: 1h 45m 34s
271:	learn: 1.0054632	test: 0.8826969	best: 0.8826969 (271)	total: 24.7s	remaining: 1h 45m 34s
272:	learn: 1.0052483	test: 0.8825883	best: 0.8825883 (272)	total: 24.8s	remaining: 1h 45m 35s
273:	learn: 1.0050264	test: 0.8824652	best: 0.8824652 (273)	total: 24.9s	remaining: 1h 45m 34s
274:	learn: 1.0048044	test: 0.8823470	best: 0.8823470 (274)	total: 25s	remaining: 1h 45m 33s
275:	learn: 1.0045857	test: 0.8822271	best: 0.882227

353:	learn: 0.9883717	test: 0.8737912	best: 0.8737912 (353)	total: 32.5s	remaining: 1h 46m 37s
354:	learn: 0.9881800	test: 0.8737015	best: 0.8737015 (354)	total: 32.6s	remaining: 1h 46m 37s
355:	learn: 0.9879843	test: 0.8735918	best: 0.8735918 (355)	total: 32.7s	remaining: 1h 46m 37s
356:	learn: 0.9877912	test: 0.8734989	best: 0.8734989 (356)	total: 32.8s	remaining: 1h 46m 37s
357:	learn: 0.9875877	test: 0.8733956	best: 0.8733956 (357)	total: 32.9s	remaining: 1h 46m 40s
358:	learn: 0.9873937	test: 0.8732835	best: 0.8732835 (358)	total: 33s	remaining: 1h 46m 41s
359:	learn: 0.9871918	test: 0.8731845	best: 0.8731845 (359)	total: 33.1s	remaining: 1h 46m 45s
360:	learn: 0.9869957	test: 0.8730834	best: 0.8730834 (360)	total: 33.2s	remaining: 1h 46m 47s
361:	learn: 0.9868020	test: 0.8729822	best: 0.8729822 (361)	total: 33.3s	remaining: 1h 46m 47s
362:	learn: 0.9866107	test: 0.8728916	best: 0.8728916 (362)	total: 33.4s	remaining: 1h 46m 46s
363:	learn: 0.9864174	test: 0.8727962	best: 0.872796

...

4629:	learn: 0.8063421	test: 0.7868119	best: 0.7868083 (4625)	total: 8m 10s	remaining: 1h 55m 18s
4630:	learn: 0.8063355	test: 0.7868112	best: 0.7868083 (4625)	total: 8m 10s	remaining: 1h 55m 18s
4631:	learn: 0.8063293	test: 0.7868109	best: 0.7868083 (4625)	total: 8m 10s	remaining: 1h 55m 18s
4632:	learn: 0.8063215	test: 0.7868056	best: 0.7868056 (4632)	total: 8m 10s	remaining: 1h 55m 18s
4633:	learn: 0.8063066	test: 0.7867993	best: 0.7867993 (4633)	total: 8m 10s	remaining: 1h 55m 17s
4634:	learn: 0.8062944	test: 0.7867992	best: 0.7867992 (4634)	total: 8m 10s	remaining: 1h 55m 18s
4635:	learn: 0.8062864	test: 0.7867980	best: 0.7867980 (4635)	total: 8m 10s	remaining: 1h 55m 18s
4636:	learn: 0.8062792	test: 0.7867986	best: 0.7867980 (4635)	total: 8m 10s	remaining: 1h 55m 17s
4637:	learn: 0.8062658	test: 0.7867922	best: 0.7867922 (4637)	total: 8m 10s	remaining: 1h 55m 17s
4638:	learn: 0.8062575	test: 0.7867915	best: 0.7867915 (4638)	total: 8m 10s	remaining: 1h 55m 17s
4639:	learn: 0.80624

4713:	learn: 0.8054593	test: 0.7866149	best: 0.7866081 (4712)	total: 8m 19s	remaining: 1h 55m 15s
4714:	learn: 0.8054459	test: 0.7866145	best: 0.7866081 (4712)	total: 8m 19s	remaining: 1h 55m 15s
4715:	learn: 0.8054335	test: 0.7866205	best: 0.7866081 (4712)	total: 8m 19s	remaining: 1h 55m 15s
4716:	learn: 0.8054234	test: 0.7866274	best: 0.7866081 (4712)	total: 8m 19s	remaining: 1h 55m 14s
4717:	learn: 0.8054154	test: 0.7866266	best: 0.7866081 (4712)	total: 8m 19s	remaining: 1h 55m 14s
4718:	learn: 0.8054095	test: 0.7866267	best: 0.7866081 (4712)	total: 8m 19s	remaining: 1h 55m 14s
4719:	learn: 0.8053962	test: 0.7866208	best: 0.7866081 (4712)	total: 8m 19s	remaining: 1h 55m 14s
4720:	learn: 0.8053885	test: 0.7866198	best: 0.7866081 (4712)	total: 8m 20s	remaining: 1h 55m 14s
4721:	learn: 0.8053763	test: 0.7866207	best: 0.7866081 (4712)	total: 8m 20s	remaining: 1h 55m 14s
4722:	learn: 0.8053677	test: 0.7866013	best: 0.7866013 (4722)	total: 8m 20s	remaining: 1h 55m 14s
4723:	learn: 0.80536

4798:	learn: 0.8045903	test: 0.7864099	best: 0.7864015 (4794)	total: 8m 28s	remaining: 1h 55m 12s
4799:	learn: 0.8045782	test: 0.7864164	best: 0.7864015 (4794)	total: 8m 28s	remaining: 1h 55m 12s
4800:	learn: 0.8045685	test: 0.7864165	best: 0.7864015 (4794)	total: 8m 29s	remaining: 1h 55m 12s
4801:	learn: 0.8045523	test: 0.7864023	best: 0.7864015 (4794)	total: 8m 29s	remaining: 1h 55m 12s
4802:	learn: 0.8045437	test: 0.7864033	best: 0.7864015 (4794)	total: 8m 29s	remaining: 1h 55m 12s
4803:	learn: 0.8045336	test: 0.7864105	best: 0.7864015 (4794)	total: 8m 29s	remaining: 1h 55m 12s
4804:	learn: 0.8045234	test: 0.7864018	best: 0.7864015 (4794)	total: 8m 29s	remaining: 1h 55m 12s
4805:	learn: 0.8045176	test: 0.7864013	best: 0.7864013 (4805)	total: 8m 29s	remaining: 1h 55m 12s
4806:	learn: 0.8045045	test: 0.7864033	best: 0.7864013 (4805)	total: 8m 29s	remaining: 1h 55m 12s
4807:	learn: 0.8044904	test: 0.7864045	best: 0.7864013 (4805)	total: 8m 29s	remaining: 1h 55m 12s
4808:	learn: 0.80447

4883:	learn: 0.8036900	test: 0.7863210	best: 0.7863184 (4881)	total: 8m 38s	remaining: 1h 55m 9s
4884:	learn: 0.8036765	test: 0.7863155	best: 0.7863155 (4884)	total: 8m 38s	remaining: 1h 55m 9s
4885:	learn: 0.8036668	test: 0.7863119	best: 0.7863119 (4885)	total: 8m 38s	remaining: 1h 55m 9s
4886:	learn: 0.8036554	test: 0.7863118	best: 0.7863118 (4886)	total: 8m 38s	remaining: 1h 55m 10s
4887:	learn: 0.8036477	test: 0.7863024	best: 0.7863024 (4887)	total: 8m 38s	remaining: 1h 55m 10s
4888:	learn: 0.8036354	test: 0.7863102	best: 0.7863024 (4887)	total: 8m 38s	remaining: 1h 55m 10s
4889:	learn: 0.8036233	test: 0.7863178	best: 0.7863024 (4887)	total: 8m 38s	remaining: 1h 55m 10s
4890:	learn: 0.8036163	test: 0.7863135	best: 0.7863024 (4887)	total: 8m 39s	remaining: 1h 55m 10s
4891:	learn: 0.8036066	test: 0.7863207	best: 0.7863024 (4887)	total: 8m 39s	remaining: 1h 55m 9s
4892:	learn: 0.8035971	test: 0.7863279	best: 0.7863024 (4887)	total: 8m 39s	remaining: 1h 55m 9s
4893:	learn: 0.8035897	te

4968:	learn: 0.8028775	test: 0.7862440	best: 0.7862440 (4968)	total: 8m 47s	remaining: 1h 55m 6s
4969:	learn: 0.8028621	test: 0.7862417	best: 0.7862417 (4969)	total: 8m 47s	remaining: 1h 55m 6s
4970:	learn: 0.8028514	test: 0.7862383	best: 0.7862383 (4970)	total: 8m 47s	remaining: 1h 55m 6s
4971:	learn: 0.8028387	test: 0.7862427	best: 0.7862383 (4970)	total: 8m 48s	remaining: 1h 55m 6s
4972:	learn: 0.8028292	test: 0.7862391	best: 0.7862383 (4970)	total: 8m 48s	remaining: 1h 55m 6s
4973:	learn: 0.8028217	test: 0.7862381	best: 0.7862381 (4973)	total: 8m 48s	remaining: 1h 55m 6s
4974:	learn: 0.8028089	test: 0.7862374	best: 0.7862374 (4974)	total: 8m 48s	remaining: 1h 55m 6s
4975:	learn: 0.8027971	test: 0.7862436	best: 0.7862374 (4974)	total: 8m 48s	remaining: 1h 55m 6s
4976:	learn: 0.8027889	test: 0.7862414	best: 0.7862374 (4974)	total: 8m 48s	remaining: 1h 55m 6s
4977:	learn: 0.8027789	test: 0.7862386	best: 0.7862374 (4974)	total: 8m 48s	remaining: 1h 55m 6s
4978:	learn: 0.8027727	test: 0

In [7]:
#training.to_pickle("pickled/training")
#training = pd.read_pickle("pickled/training")

#pickle.dump(cb_model, open( "pickled/cb_model", "wb"), protocol=4)

cb_model = pickle.load( open( "pickled/cb_model", "rb" ) )

In [10]:
np.array(cb_features)[np.argsort(cb_model.get_feature_importance())[::-1]]

array(['item_block_units_lag_1', 'shop_category_units',
       'item_share_of_total_units', 'shop_first_two_blocks_units',
       'shop_share_of_units', 'item_mean_price_block_lag_2',
       'item_first_two_blocks_units', 'shop_max_turnover_block',
       'item_number_of_consecutive_days_with_activity', 'item_units',
       'shop_category_turnover', 'shop_max_units_block',
       'item_last_two_blocks_units', 'item_days_of_activity',
       'item_mean_units_block', 'shop_block_units_lag_1',
       'category_mean_turnover_day', '12', 'item_first_day',
       'shop_min_turnover_block', 'shop_block_turnover_lag_1',
       'item_block_turnover_lag_1', 'item_first_block',
       'item_block_units_lag_2', 'shop_category_max_units_block',
       'subcategory_block_units_lag_1', '11',
       'item_mean_price_block_lag_1', 'area_mean_price_block_lag_1',
       'item_block_turnover_lag_2',
       'shop_fluctuation_units_first_last_blocks',
       'shop_category_max_turnover_block', 'shop_turnove

In [13]:
xg_features = np.array(cb_features)[np.argsort(cb_model.get_feature_importance())[::-1]][0:31]
xg_features

array(['item_block_units_lag_1', 'shop_category_units',
       'item_share_of_total_units', 'shop_first_two_blocks_units',
       'shop_share_of_units', 'item_mean_price_block_lag_2',
       'item_first_two_blocks_units', 'shop_max_turnover_block',
       'item_number_of_consecutive_days_with_activity', 'item_units',
       'shop_category_turnover', 'shop_max_units_block',
       'item_last_two_blocks_units', 'item_days_of_activity',
       'item_mean_units_block', 'shop_block_units_lag_1',
       'category_mean_turnover_day', '12', 'item_first_day',
       'shop_min_turnover_block', 'shop_block_turnover_lag_1',
       'item_block_turnover_lag_1', 'item_first_block',
       'item_block_units_lag_2', 'shop_category_max_units_block',
       'subcategory_block_units_lag_1', '11',
       'item_mean_price_block_lag_1', 'area_mean_price_block_lag_1',
       'item_block_turnover_lag_2',
       'shop_fluctuation_units_first_last_blocks'], dtype='<U47')

In [ ]:
gc.collect()
params =   {
    'objective' : 'reg:linear',
    #'tree_method':'gpu_hist',
    #'gpu_id': 0,
    'learning_rate': 0.001, 
    #'gamma' : 0.3, 
    #'min_child_weight' : 3,
    #'nthread' : 16,
    #'max_depth' : 30,
    #'subsample' : 0.9, 
    #'colsample_bytree' : 0.8, 
    'seed':42, 
    'eval_metric' : "rmse",
    'num_boost_round' : 300,
    #'n_estimators':999,
    #'max_leaves': 300
}


tr_data = xgb.DMatrix(x_train[xg_features], y_train)
va_data = xgb.DMatrix(x_val[xg_features], y_val)
watchlist = [(tr_data, 'train'), (va_data, 'valid')]

xg_model = xgb.train(params, tr_data, 300, watchlist, maximize=False, early_stopping_rounds = 30, verbose_eval=True)

In [ ]:
gc.collect()
lgtrain = lgbm.Dataset(x_train[cb_features], label=y_train)
lgval = lgbm.Dataset(x_val[cb_features], label=y_val)



#[0.00542047893814942, 29, 24, 0.39949465609514856, 1, 0.67943500, 10]
params = {
        "num_threads": 16,
        "verbosity": -1,
        #"zero_as_missing": "true",
        "boosting":'gbdt',
        "objective" : "regression",
        "metric" : "rmse",
        "seed": 42,
        #"max_bin": 10,#default 255
        #"num_leaves": 10, #default 31
        #"bagging_fraction": 0.7,
        #"bagging_freq": 1,
        #"min_data_in_leaf": 50000,
        #"feature_fraction": 0.7,
        #"lambda_l2": 3,
        #"max_depth": 2,
        #"min_gain_to_split": 10,
        "learning_rate" : 0.001,
        "histogram_pool_size": 1000,
        #"categorical_column": [0,1,2,3,4]
}

evals_result = {}
model_lgb = lgbm.train(params, lgtrain, 1000, 
                      valid_sets=[lgval], 
                      early_stopping_rounds=100, 
                      verbose_eval=100, 
                      evals_result=evals_result)



In [27]:
pd.set_option('display.max_columns', None)  
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', -1)
x_train[cb_features].sample(10)

,item_first_block,item_last_block,is_first_two_blocks,is_last_two_blocks,item_units,item_mean_units_block,item_mean_units_day,item_max_units_block,item_min_units_block,item_max_units_day,item_min_units_day,item_turnover,item_mean_turnover_block,item_mean_turnover_day,item_max_turnover_block,item_min_turnover_block,item_max_turnover_day,item_min_turnover_day,item_days_of_activity,item_blocks_of_activity,item_days_since_start,item_mean_day_between_activity,item_longest_stretch_days_without_activity,item_longest_stretch_blocks_without_activity,item_longest_stretch_block_with_activity,item_number_of_consecutive_days_with_activity,item_days_between_start_and_first_activity,item_blocks_between_start_and_first_activity,item_first_day,item_last_day,item_activity_on_all_blocks,item_mean_price,item_min_price,item_max_price,item_number_different_prices,item_price_amplitude,item_deviation_mean_category_price,item_first_two_blocks_units,item_last_two_blocks_units,item_fluctuation_units_first_last_blocks,item_first_two_blocks_mean_price,item_last_two_blocks_mean_price,item_fluctuation_price_first_last_blocks,item_share_of_total_units,item_share_of_total_turnover,category_units,category_mean_units_block,category_mean_units_day,category_max_units_block,category_min_units_block,category_max_units_day,category_min_units_day,category_turnover,category_mean_turnover_block,category_mean_turnover_day,category_max_turnover_block,category_min_turnover_block,category_max_turnover_day,category_min_turnover_day,category_mean_price,category_min_price,category_max_price,category_first_two_blocks_units,category_last_two_blocks_units,category_fluctuation_units_first_last_blocks,category_first_two_blocks_mean_price,category_last_two_blocks_mean_price,category_fluctuation_price_first_last_blocks,video_game,gaming_old_gen,gaming_new_gen,pc_games,payment_cards,movies,movies_niche,books,music,music_CD,music_vinyl,gifts,software,subcategory_units,subcategory_mean_units_block,subcategory_mean_units_day,subcategory_max_units_block,subcategory_min_units_block,subcategory_max_units_day,subcategory_min_units_day,subcategory_turnover,subcategory_mean_turnover_block,subcategory_mean_turnover_day,subcategory_max_turnover_block,subcategory_min_turnover_block,subcategory_max_turnover_day,subcategory_min_turnover_day,category_share_of_total_units,category_share_of_total_turnover,subcategory_share_of_total_units,subcategory_share_of_total_turnover,subcategory_first_two_blocks_units,subcategory_last_two_blocks_units,subcategory_fluctuation_units_first_last_blocks,subcategory_first_two_blocks_mean_price,subcategory_last_two_blocks_mean_price,subcategory_fluctuation_price_first_last_blocks,shop_units,shop_mean_units_block,shop_mean_units_day,shop_max_units_block,shop_min_units_block,shop_max_units_day,shop_min_units_day,shop_turnover,shop_mean_turnover_block,shop_mean_turnover_day,shop_max_turnover_block,shop_min_turnover_block,shop_max_turnover_day,shop_min_turnover_day,shop_mean_price,shop_first_two_blocks_units,shop_last_two_blocks_units,shop_fluctuation_units_first_last_blocks,shop_first_two_blocks_mean_price,shop_last_two_blocks_mean_price,shop_fluctuation_price_first_last_blocks,shop_share_of_units,shop_share_of_turnover,shop_TC,shop_TRK,shop_SEC,shop_shopping_center,shop_moscow,max_category_units,max_category_turnover,area_units,area_mean_units_block,area_mean_units_day,area_max_units_block,area_min_units_block,area_max_units_day,area_min_units_day,area_turnover,area_mean_turnover_block,area_mean_turnover_day,area_max_turnover_block,area_min_turnover_block,area_max_turnover_day,area_min_turnover_day,area_mean_price,area_first_two_blocks_units,area_last_two_blocks_units,area_fluctuation_units_first_last_blocks,area_first_two_blocks_mean_price,area_last_two_blocks_mean_price,area_fluctuation_price_first_last_blocks,shop_category_units,shop_category_mean_units_block,shop_category_mean_units_day,shop_category_max_units_block,shop_category_min_units_block,shop_category_max_un

In [ ]:
tes